# Imports

In [ ]:
import pandas as pd
import jax
import jax.numpy as jnp
import json
import os
import matplotlib.pyplot as plt
import random
import numpy as np
import seaborn as sns

import scipy.stats as stats
from IPython.display import HTML, display, Image
from IPython.display import display
from scipy.optimize import curve_fit
from scipy.stats import binned_statistic
from sklearn.metrics import r2_score

sns.set(style="white", context="talk")

#for bayesian model
!pip install pymc arviz --quiet
import pymc as pm
import arviz as az
from matplotlib.ticker import MaxNLocator
from scipy.special import expit
from scipy.stats import linregress
from sklearn.metrics import roc_auc_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
tmp_dir = "/content/drive/MyDrive/"
drive.mount("/content/drive", force_remount=True)

In [ ]:
the_dir = "/content/drive/MyDrive/files_for_NMI_submission_R1/"
the_old_dir = "/content/drive/MyDrive/"

# Functions

In [ ]:
#Find NaNs and get rid of those rows

def find_nan_in_df(df):
    """
    Searches a Pandas DataFrame for NaN values and reports their locations.

    Args:
        df: The Pandas DataFrame to search.

    Returns:
        None. Prints information about found NaN values.
    """
    if df.isna().any().any():

        nan_mask = df.isna()
        total_nans = df.isna().sum().sum()
        rows_with_nan = df[df.isna().any(axis=1)]
        num_rows_with_nan = len(rows_with_nan)
        df_no_nans = df.dropna()
    else:

        df_no_nans = df  # <-- ADD THIS LINE

    return df_no_nans

In [ ]:
def add_new_confidence_columns(df, expt_type):

    # Rename columns
    df = df.rename(columns={
        "change_of_mind_actual": "change_of_mind",
        "other_llm_answer_shown": "other_llm_answer",
        "other_LLM_answer": "other_llm_answer",
        "other_LLM_accuracy": "other_llm_accuracy",
        "initial_confidence_a": "confidence_a",
        "initial_confidence_b": "confidence_b"
    })

    def get_initial_confidence_chosen(row):
        if row['initial_answer'] == 'a':
            return row['confidence_a']
        elif row['initial_answer'] == 'b':
            return row['confidence_b']
        else:
            return np.nan

    def get_initial_confidence_final_chosen(row):
        if row['final_answer'] == 'a':
            return row['confidence_a']
        elif row['final_answer'] == 'b':
            return row['confidence_b']
        else:
            return np.nan

    def get_confidence_difference(row):
        if row['initial_answer'] == 'a':
            return row['second_turn_confidence_a'] - row['initial_confidence_chosen']
        elif row['initial_answer'] == 'b':
            return row['second_turn_confidence_b'] - row['initial_confidence_chosen']
        else:
            return np.nan

    def get_second_turn_confidence_initial_chosen_lat(row):
        if row['initial_answer'] == 'a':
            return row['second_turn_confidence_a']
        elif row['initial_answer'] == 'b':
            return row['second_turn_confidence_b']
        else:
            return np.nan

    def get_second_turn_confidence_final_chosen_lat(row):
        if row['final_answer'] == 'a':
            return row['second_turn_confidence_a']
        elif row['final_answer'] == 'b':
            return row['second_turn_confidence_b']
        else:
            return np.nan

    # Add new columns
    df['initial_confidence_chosen'] = df.apply(get_initial_confidence_chosen, axis=1)
    df['initial_confidence_final_chosen'] = df.apply(get_initial_confidence_final_chosen, axis=1)
    df['confidence_difference'] = df.apply(get_confidence_difference, axis=1)
    df['second_turn_binary_correctness'] = (df['final_answer'] == df['gt_answer']).astype(int)
    df['second_turn_confidence_initial_chosen'] = df.apply(get_second_turn_confidence_initial_chosen_lat, axis=1)
    df['second_turn_confidence_final_chosen'] = df.apply(get_second_turn_confidence_final_chosen_lat, axis=1)
    df['confidence_gap_initial'] = df['second_turn_confidence_initial_chosen'] - df['initial_confidence_chosen']
    df['confidence_gap_initial_chosen'] = df['second_turn_confidence_initial_chosen'] - df['initial_confidence_chosen']

    # Answer Wrong -specific columns
    if expt_type == 'hog':
        # Set second_turn_confidence_hog_chosen based on hog answer
        df['second_turn_confidence_hog_chosen'] = df.apply(
            lambda row: row['second_turn_confidence_a'] if row['initial_answer_hog'] == 'a' else row['second_turn_confidence_b'],
            axis=1
        )

        def get_initial_confidence_hog_chosen(row):
            if row['initial_answer_hog'] == 'a':
                return row['confidence_a']
            elif row['initial_answer_hog'] == 'b':
                return row['confidence_b']
            else:
                return np.nan

        df['initial_confidence_hog_chosen'] = df.apply(get_initial_confidence_hog_chosen, axis=1)
        df['confidence_gap_hog_chosen'] = df['second_turn_confidence_hog_chosen'] - df['initial_confidence_hog_chosen']

    return df

In [ ]:
def add_new_confidence_columns_log_odds_space(df, expt_type):

    # Convert from probability to log odds if needed
    epsilon = 1e-9
    for col in ['confidence_a', 'confidence_b', 'second_turn_confidence_a', 'second_turn_confidence_b']:
        clipped_prob = df[col].clip(lower=epsilon, upper=1-epsilon)
        df[f'{col}_log_odds'] = np.log(clipped_prob / (1 - clipped_prob))


    def get_confidence_for_answer(row, answer_col, conf_a_col, conf_b_col):
        """Get confidence value based on answer choice (a or b)"""
        if row[answer_col] == 'a':
            return row[conf_a_col]
        elif row[answer_col] == 'b':
            return row[conf_b_col]
        else:
            return np.nan

    # 1. Initial confidence for chosen answer
    df['initial_confidence_chosen_log_odds'] = df.apply(
        lambda row: get_confidence_for_answer(
            row, 'initial_answer',
            'confidence_a_log_odds',
            'confidence_b_log_odds'
        ), axis=1
    )

    # 2. Initial confidence for final chosen answer
    df['initial_confidence_final_chosen_log_odds'] = df.apply(
        lambda row: get_confidence_for_answer(
            row, 'final_answer',
            'confidence_a_log_odds',
            'confidence_b_log_odds'
        ), axis=1
    )

    # 3. Second turn confidence for initially chosen option
    df['second_turn_confidence_initial_chosen_log_odds'] = df.apply(
        lambda row: get_confidence_for_answer(
            row, 'initial_answer',
            'second_turn_confidence_a_log_odds',
            'second_turn_confidence_b_log_odds'
        ), axis=1
    )

    # 4. Second turn confidence for final chosen answer
    df['second_turn_confidence_final_chosen_log_odds'] = df.apply(
        lambda row: get_confidence_for_answer(
            row, 'final_answer',
            'second_turn_confidence_a_log_odds',
            'second_turn_confidence_b_log_odds'
        ), axis=1
    )

    # 5. Confidence difference (change in confidence for initially chosen option)
    df['confidence_difference_log_odds'] = (
        df['second_turn_confidence_initial_chosen_log_odds'] -
        df['initial_confidence_chosen_log_odds']
    )

    # 6. Confidence gap (same as confidence_difference - redundant but keeping for compatibility)
    df['confidence_gap_initial_log_odds'] = (
        df['second_turn_confidence_initial_chosen_log_odds'] -
        df['initial_confidence_chosen_log_odds']
    )

    # 7. Another redundant column with same calculation
    df['confidence_gap_initial_chosen_log_odds'] = (
        df['second_turn_confidence_initial_chosen_log_odds'] -
        df['initial_confidence_chosen_log_odds']
    )

    # Conditional columns for answer wrong experiment type
    if expt_type == 'hog':
        # Initial confidence for hog chosen answer
        df['initial_confidence_hog_chosen_log_odds'] = df.apply(
            lambda row: get_confidence_for_answer(
                row, 'initial_answer_hog',
                'confidence_a_log_odds',
                'confidence_b_log_odds'
            ), axis=1
        )

        # Second turn confidence for hog chosen answer
        df['second_turn_confidence_hog_chosen_log_odds'] = df.apply(
            lambda row: get_confidence_for_answer(
                row, 'initial_answer_hog',
                'second_turn_confidence_a_log_odds',
                'second_turn_confidence_b_log_odds'
            ), axis=1
        )

        # Confidence gap for hog chosen answer
        df['confidence_gap_hog_chosen_log_odds'] = (
            df['second_turn_confidence_hog_chosen_log_odds'] -
            df['initial_confidence_hog_chosen_log_odds']
        )

    return df

In [ ]:
def add_new_confidence_columns_4_choice(df, expt_type="standard"):
    import pandas as pd
    import numpy as np

    df = df.copy()

    df = df.rename(columns={"change_of_mind_actual": "change_of_mind"})
    df = df.rename(columns={"other_llm_answer_shown": "other_llm_answer"})
    df = df.rename(columns={"other_LLM_answer": "other_llm_answer"})
    df = df.rename(columns={"other_LLM_accuracy": "other_llm_accuracy"})

    for i in range(1, 5):
        if f"initial_confidence_{i}" in df.columns:
            df = df.rename(columns={f"initial_confidence_{i}": f"confidence_{i}"})

    def safe_int_convert(value):
        if pd.isna(value):
            return None
        try:
            return int(float(value))
        except:
            return None

    def get_initial_confidence_chosen(row):
        answer = safe_int_convert(row['initial_answer'])
        if answer in [1, 2, 3, 4]:
            col_name = f'confidence_{answer}'
            if col_name in row.index:
                return row[col_name]
        return np.nan

    def get_initial_confidence_final_chosen(row):
        answer = safe_int_convert(row['final_answer'])
        if answer in [1, 2, 3, 4]:
            col_name = f'confidence_{answer}'
            if col_name in row.index:
                return row[col_name]
        return np.nan

    def get_second_turn_confidence_initial_chosen(row):
        answer = safe_int_convert(row['initial_answer'])
        if answer in [1, 2, 3, 4]:
            col_name = f'second_turn_confidence_{answer}'
            if col_name in row.index:
                return row[col_name]
        return np.nan

    def get_second_turn_confidence_final_chosen(row):
        answer = safe_int_convert(row['final_answer'])
        if answer in [1, 2, 3, 4]:
            col_name = f'second_turn_confidence_{answer}'
            if col_name in row.index:
                return row[col_name]
        return np.nan

    def get_confidence_difference(row):
        initial_conf = row['initial_confidence_chosen']
        second_conf = row['second_turn_confidence_initial_chosen']
        if pd.notna(initial_conf) and pd.notna(second_conf):
            return second_conf - initial_conf
        return np.nan

    df['initial_confidence_chosen'] = df.apply(get_initial_confidence_chosen, axis=1)
    df['initial_confidence_final_chosen'] = df.apply(get_initial_confidence_final_chosen, axis=1)
    df['second_turn_confidence_initial_chosen'] = df.apply(get_second_turn_confidence_initial_chosen, axis=1)
    df['second_turn_confidence_final_chosen'] = df.apply(get_second_turn_confidence_final_chosen, axis=1)
    df['confidence_difference'] = df.apply(get_confidence_difference, axis=1)
    df['confidence_gap_initial'] = df['confidence_difference']
    df['confidence_gap_initial_chosen'] = df['confidence_difference']

    df['second_turn_binary_correctness'] = (df['final_answer'] == df['gt_answer']).astype(int)

    if expt_type == 'hog':
        def get_second_turn_confidence_hog(row):
            hog_answer = safe_int_convert(row['initial_answer_hog'])
            if hog_answer in [1, 2, 3, 4]:
                return row[f'second_turn_confidence_{hog_answer}']
            return np.nan

        def get_initial_confidence_hog_chosen(row):
            answer = safe_int_convert(row['initial_answer_hog'])
            if answer in [1, 2, 3, 4]:
                return row[f'confidence_{answer}']
            return np.nan

        df['second_turn_confidence_hog_chosen'] = df.apply(get_second_turn_confidence_hog, axis=1)
        df['initial_confidence_hog_chosen'] = df.apply(get_initial_confidence_hog_chosen, axis=1)
        df['confidence_gap_hog_chosen'] = df['second_turn_confidence_hog_chosen'] - df['initial_confidence_hog_chosen']

    return df

In [ ]:
def add_bayes_optimal_columns(df, n_alt_choices=1):

    def calculate_bayes_probability(initial_confidence, initial_answer, final_answer,
                                    other_llm_answer_type, other_llm_accuracy,
                                    expt_type, other_llm_answer=None, initial_answer_hog=None):
        # Normalize prior confidence
        probabilities = initial_confidence.copy()
        prob_sum = sum(probabilities)
        probabilities = [p / prob_sum for p in probabilities] if prob_sum > 0 else [0.5, 0.5]

        accuracy = other_llm_accuracy  # should be in [0, 1]

        # Determine advisor's recommendation
        if other_llm_answer_type == "Same":
            advisor_recommendation = initial_answer_hog if expt_type == "hog" else initial_answer
            likelihood_given_advice = [
                accuracy if i == advisor_recommendation else (1 - accuracy) for i in range(2)
            ]

        elif other_llm_answer_type == "Opposite":
            if other_llm_answer is None:
                advisor_recommendation = 1 - (initial_answer_hog if expt_type == "hog" else initial_answer)
            else:
                advisor_recommendation = other_llm_answer
            likelihood_given_advice = [
                accuracy if i == advisor_recommendation else (1 - accuracy) for i in range(2)
            ]

        elif other_llm_answer_type == "Nothing":
            posterior_probabilities = probabilities
            likelihood_given_advice = None

        else:
            raise ValueError("Invalid other_llm_answer_type. Must be 'Same', 'Opposite', or 'Nothing'.")

        # Compute Bayesian posterior
        if other_llm_answer_type != "Nothing":
            denominator = sum(likelihood_given_advice[i] * probabilities[i] for i in range(2))
            posterior_probabilities = [
                (likelihood_given_advice[i] * probabilities[i]) / denominator if denominator > 0 else 0.5
                for i in range(2)
            ]


        result = {
            "bayes_optimal_probability_a": posterior_probabilities[0],
            "bayes_optimal_probability_b": posterior_probabilities[1],
            "bayes_optimal_probability_initial_chosen": posterior_probabilities[initial_answer],
            "bayes_optimal_probability_final_chosen": posterior_probabilities[final_answer],
        }

        if expt_type == "hog" and initial_answer_hog is not None:
            result["bayes_optimal_probability_hog_chosen"] = posterior_probabilities[initial_answer_hog]

        return result

    def apply_bayes_calculation(row):
        expt_type = row['initial_answer_display']  # 'shown', 'hidden', or 'hog' (hog = answer wrong)

        # Translate answers to index: 'a' → 0, 'b' → 1
        initial_answer_index = 0 if row['initial_answer'].lower() == 'a' else 1
        final_answer_index = 0 if row['final_answer'].lower() == 'a' else 1

        if pd.notna(row['other_llm_answer']) and row['other_llm_answer_type'] == "Opposite":
            other_llm_answer_index = 0 if row['other_llm_answer'].lower() == 'a' else 1
        else:
            other_llm_answer_index = None

        if expt_type == "hog" and pd.notna(row.get('initial_answer_hog', None)):
            initial_answer_hog_index = 0 if row['initial_answer_hog'].lower() == 'a' else 1
        else:
            initial_answer_hog_index = None

        return calculate_bayes_probability(
            initial_confidence=[row['confidence_a'], row['confidence_b']],
            initial_answer=initial_answer_index,
            final_answer=final_answer_index,
            other_llm_answer_type=row['other_llm_answer_type'],
            other_llm_accuracy=row['other_llm_accuracy'] / 100.0,
            other_llm_answer=other_llm_answer_index,
            initial_answer_hog=initial_answer_hog_index,
            expt_type=expt_type
        )


    bayes_results = df.apply(apply_bayes_calculation, axis=1, result_type='expand')


    df = pd.concat([df, bayes_results], axis=1)

    return df

In [ ]:
def add_bayes_optimal_columns_4_choice(df):
    import pandas as pd
    import numpy as np

    def calculate_bayes_probability(initial_confidence, initial_answer, final_answer,
                                    other_llm_answer_type, other_llm_accuracy,
                                    other_llm_answer=None):
        probabilities = initial_confidence.copy()
        prob_sum = sum(probabilities)
        if prob_sum > 0:
            probabilities = [p / prob_sum for p in probabilities]
        else:
            probabilities = [0.25, 0.25, 0.25, 0.25]

        prob_sum = sum(probabilities)
        probabilities = [p / prob_sum for p in probabilities]

        accuracy = other_llm_accuracy

        if other_llm_answer_type == "Nothing":
            posterior_probabilities = probabilities

        elif other_llm_answer_type in ["Same", "Opposite"]:
            if other_llm_answer is None:
                raise ValueError(f"For '{other_llm_answer_type}' advice type, other_llm_answer must be provided")

            advisor_recommendation = other_llm_answer

            likelihood_given_advice = []
            for i in range(4):
                if i == advisor_recommendation:
                    likelihood_given_advice.append(accuracy)
                else:
                    likelihood_given_advice.append((1 - accuracy) / 3)

            denominator = sum(likelihood_given_advice[i] * probabilities[i] for i in range(4))

            if denominator > 0:
                posterior_probabilities = [
                    (likelihood_given_advice[i] * probabilities[i]) / denominator
                    for i in range(4)
                ]
            else:
                posterior_probabilities = [0.25, 0.25, 0.25, 0.25]

            post_sum = sum(posterior_probabilities)
            posterior_probabilities = [p / post_sum for p in posterior_probabilities]

        else:
            raise ValueError("Invalid other_llm_answer_type. Must be 'Same', 'Opposite', or 'Nothing'.")

        result = {
            "bayes_optimal_probability_1": posterior_probabilities[0],
            "bayes_optimal_probability_2": posterior_probabilities[1],
            "bayes_optimal_probability_3": posterior_probabilities[2],
            "bayes_optimal_probability_4": posterior_probabilities[3],
            "bayes_optimal_probability_initial_chosen": posterior_probabilities[initial_answer],
            "bayes_optimal_probability_final_chosen": posterior_probabilities[final_answer],
        }

        return result

    def apply_bayes_calculation(row):
        def answer_to_index(answer):
            if pd.isna(answer):
                return None
            try:
                ans = int(float(answer))
                return ans - 1 if 1 <= ans <= 4 else None
            except:
                return None

        initial_answer_index = answer_to_index(row['initial_answer'])
        final_answer_index = answer_to_index(row['final_answer'])

        if pd.notna(row.get('other_llm_answer')):
            other_llm_answer_index = answer_to_index(row['other_llm_answer'])
        else:
            other_llm_answer_index = None

        if initial_answer_index is None or final_answer_index is None:
            return {
                "bayes_optimal_probability_1": np.nan,
                "bayes_optimal_probability_2": np.nan,
                "bayes_optimal_probability_3": np.nan,
                "bayes_optimal_probability_4": np.nan,
                "bayes_optimal_probability_initial_chosen": np.nan,
                "bayes_optimal_probability_final_chosen": np.nan
            }

        initial_confidence = [
            row.get('confidence_1', 0),
            row.get('confidence_2', 0),
            row.get('confidence_3', 0),
            row.get('confidence_4', 0)
        ]

        other_llm_accuracy = row['other_llm_accuracy'] / 100.0

        try:
            return calculate_bayes_probability(
                initial_confidence=initial_confidence,
                initial_answer=initial_answer_index,
                final_answer=final_answer_index,
                other_llm_answer_type=row['other_llm_answer_type'],
                other_llm_accuracy=other_llm_accuracy,
                other_llm_answer=other_llm_answer_index
            )
        except:
            return {
                "bayes_optimal_probability_1": np.nan,
                "bayes_optimal_probability_2": np.nan,
                "bayes_optimal_probability_3": np.nan,
                "bayes_optimal_probability_4": np.nan,
                "bayes_optimal_probability_initial_chosen": np.nan,
                "bayes_optimal_probability_final_chosen": np.nan
            }

    bayes_results = df.apply(apply_bayes_calculation, axis=1, result_type='expand')
    df = pd.concat([df, bayes_results], axis=1)

    return df

In [ ]:
def add_bayes_optimal_columns_log_odds_space(df, calculate_deviations=True, print_summary=False):

    # Epsilon to avoid log(0) or division by zero
    EPSILON = 1e-9

    def calculate_bayes_log_odds(initial_log_odds_a, initial_log_odds_b,
                            initial_answer, final_answer,
                            other_llm_answer_type, other_llm_accuracy,
                            other_llm_answer=None,
                            initial_answer_hog=None,
                            expt_type=None):

        # Convert accuracy to probability (it comes as percentage 0-100)
        if other_llm_accuracy > 1:
            accuracy = other_llm_accuracy / 100.0
        else:
            accuracy = other_llm_accuracy


        accuracy = np.clip(accuracy, EPSILON, 1 - EPSILON)


        if np.abs(accuracy - 0.5) < EPSILON:
            log_likelihood_ratio = 0.0

            result = {
                "bayes_optimal_log_odds_a": initial_log_odds_a,
                "bayes_optimal_log_odds_b": initial_log_odds_b,
            }

            # Add log odds for specific choices (no change from prior)
            if initial_answer.lower() == 'a':
                result["bayes_optimal_log_odds_initial_chosen"] = initial_log_odds_a
            else:
                result["bayes_optimal_log_odds_initial_chosen"] = initial_log_odds_b

            if final_answer.lower() == 'a':
                result["bayes_optimal_log_odds_final_chosen"] = initial_log_odds_a
            else:
                result["bayes_optimal_log_odds_final_chosen"] = initial_log_odds_b

            if expt_type == "hog" and initial_answer_hog is not None:
                if initial_answer_hog.lower() == 'a':
                    result["bayes_optimal_log_odds_hog_chosen"] = initial_log_odds_a
                else:
                    result["bayes_optimal_log_odds_hog_chosen"] = initial_log_odds_b

            return result

        # Map answer letters to indices
        answer_map = {'a': 0, 'b': 1}
        initial_idx = answer_map.get(initial_answer.lower(), 0)
        final_idx = answer_map.get(final_answer.lower(), 0)

        prior_log_odds = initial_log_odds_a

        # Calculate log likelihood ratio based on advice type
        if other_llm_answer_type == "Same":
            # Advisor recommends same as initial choice (or answer wrong choice if applicable)
            if expt_type == "hog" and initial_answer_hog is not None:
                advisor_choice = initial_answer_hog.lower()
            else:
                advisor_choice = initial_answer.lower()


            if advisor_choice == 'a':

                log_likelihood_ratio = np.log(accuracy / (1 - accuracy))
            else:
                log_likelihood_ratio = np.log((1 - accuracy) / accuracy)

        elif other_llm_answer_type == "Opposite":

            if other_llm_answer is not None:
                advisor_choice = other_llm_answer.lower()
            else:

                if expt_type == "hog" and initial_answer_hog is not None:
                    advisor_choice = 'b' if initial_answer_hog.lower() == 'a' else 'a'
                else:
                    advisor_choice = 'b' if initial_answer.lower() == 'a' else 'a'


            if advisor_choice == 'a':
                log_likelihood_ratio = np.log(accuracy / (1 - accuracy))
            else:
                log_likelihood_ratio = np.log((1 - accuracy) / accuracy)

        elif other_llm_answer_type == "Nothing" or other_llm_answer_type == "Neutral":

            log_likelihood_ratio = 0

        else:
            raise ValueError(f"Invalid other_llm_answer_type: {other_llm_answer_type}")

        # Calculate posterior log odds
        posterior_log_odds_a = prior_log_odds + log_likelihood_ratio
        posterior_log_odds_b = -posterior_log_odds_a  # Since log(P(b)/P(a)) = -log(P(a)/P(b))


        result = {
            "bayes_optimal_log_odds_a": posterior_log_odds_a,
            "bayes_optimal_log_odds_b": posterior_log_odds_b,
        }


        if initial_answer.lower() == 'a':
            result["bayes_optimal_log_odds_initial_chosen"] = posterior_log_odds_a
        else:
            result["bayes_optimal_log_odds_initial_chosen"] = posterior_log_odds_b

        if final_answer.lower() == 'a':
            result["bayes_optimal_log_odds_final_chosen"] = posterior_log_odds_a
        else:
            result["bayes_optimal_log_odds_final_chosen"] = posterior_log_odds_b

        # Handle Answer Wrong experiment type
        if expt_type == "hog" and initial_answer_hog is not None:
            if initial_answer_hog.lower() == 'a':
                result["bayes_optimal_log_odds_hog_chosen"] = posterior_log_odds_a
            else:
                result["bayes_optimal_log_odds_hog_chosen"] = posterior_log_odds_b

        return result

    def apply_bayes_log_odds_calculation(row):

        expt_type = row.get('initial_answer_display', 'shown')

        # Get other_llm_answer if provided and advice type is Opposite
        other_llm_answer = None
        if pd.notna(row.get('other_llm_answer')) and row['other_llm_answer_type'] == "Opposite":
            other_llm_answer = row['other_llm_answer']

        # Get initial_answer_hog for Answer Wrong
        initial_answer_hog = None
        if expt_type == "hog" and pd.notna(row.get('initial_answer_hog')):
            initial_answer_hog = row['initial_answer_hog']

        return calculate_bayes_log_odds(
            initial_log_odds_a=row['confidence_a_log_odds'],
            initial_log_odds_b=row['confidence_b_log_odds'],
            initial_answer=row['initial_answer'],
            final_answer=row['final_answer'],
            other_llm_answer_type=row['other_llm_answer_type'],
            other_llm_accuracy=row['other_llm_accuracy'],
            other_llm_answer=other_llm_answer,
            initial_answer_hog=initial_answer_hog,
            expt_type=expt_type
        )


    bayes_results = df.apply(apply_bayes_log_odds_calculation, axis=1, result_type='expand')


    df = pd.concat([df, bayes_results], axis=1)

    if print_summary:
        print("Created Bayesian log odds columns:")
        bayes_columns = [col for col in bayes_results.columns]
        for col in bayes_columns:
            print(f"  - {col}")

    if calculate_deviations:

        df['log_odds_deviation_initial'] = (
            df['second_turn_confidence_initial_chosen_log_odds'] -
            df['bayes_optimal_log_odds_initial_chosen']
        )

        df['log_odds_deviation_final'] = (
            df['second_turn_confidence_final_chosen_log_odds'] -
            df['bayes_optimal_log_odds_final_chosen']
        )

    return df

# Functions For Bayesian Model

In [ ]:
def preprocess_experiment_data(df):

    def find_nan_in_df(df):

        if df.isna().any().any():

            total_nans = df.isna().sum().sum()
            rows_with_nan = df[df.isna().any(axis=1)]
            num_rows_with_nan = len(rows_with_nan)



            df_no_nans = df.dropna()

            return df_no_nans
        else:

            return df

    def get_first_turn_confidence_initial_chosen(row):
        if row['initial_answer'] in ['1', '2', '3', '4']:
            return row[f'confidence_{row["initial_answer"]}']
        else:
            return np.nan

    def get_second_turn_confidence_initial_chosen(row):
        if row['initial_answer'] in ['1', '2', '3', '4']:
            return row[f'second_turn_confidence_{row["initial_answer"]}']
        else:
            return np.nan

    def get_first_turn_confidence_final_chosen(row):
        if row['final_answer'] in ['1', '2', '3', '4']:
            return row[f'confidence_{row["final_answer"]}']
        else:
            return np.nan

    def get_second_turn_confidence_final_chosen(row):
        if row['final_answer'] in ['1', '2', '3', '4']:
            return row[f'second_turn_confidence_{row["final_answer"]}']
        else:
            return np.nan

    df = find_nan_in_df(df)

    invalid_initial_answers = df[~df['initial_answer'].isin(['1', '2', '3', '4'])]

    df['first_turn_confidence_initial_chosen'] = df.apply(get_first_turn_confidence_initial_chosen, axis=1)
    df['second_turn_confidence_initial_chosen'] = df.apply(get_second_turn_confidence_initial_chosen, axis=1)
    df['first_turn_confidence_final_chosen'] = df.apply(get_first_turn_confidence_final_chosen, axis=1)
    df['second_turn_confidence_final_chosen'] = df.apply(get_second_turn_confidence_final_chosen, axis=1)

    df['second_turn_binary_correctness'] = (df['final_answer'] == df['gt_answer']).astype(int)


    direction_mapping = {
        "Same": 1,
        "Opposite": -1,
        "Nothing": 0
    }
    df['advice_direction'] = df['other_llm_answer_type'].replace(direction_mapping)


    confirmation_mapping = {
        "Same": 1,
        "Opposite": 0,
        "Nothing": 0
    }
    df['confirmation_flag'] = df['other_llm_answer_type'].replace(confirmation_mapping)


    shown_mapping = {
        "shown": 1,
        "hidden": 0
    }
    df['shown_flag'] = df['initial_answer_display'].replace(shown_mapping)


    df["final_answer"] = df["final_answer"].astype(int)
    df["initial_answer"] = df["initial_answer"].astype(int)

    if 'other_LLM_answer' in df.columns:
        df.rename(columns={'other_LLM_answer': 'other_llm_answer'}, inplace=True)

    df["other_llm_answer"] = df["other_llm_answer"].apply(
        lambda x: -1 if isinstance(x, str) and "hidden from" in x else x
    )

    df["other_llm_answer"] = pd.to_numeric(df["other_llm_answer"], errors="coerce")

    df['other_llm_accuracy'] = df['other_llm_accuracy'] / 100  # RESCALE from percentage

    epsilon_acc = 1e-9
    df["other_llm_accuracy_capped"] = np.clip(
        df["other_llm_accuracy"],
        a_min=epsilon_acc,
        a_max=1 - epsilon_acc
    )


    df['effective_other_llm_accuracy_capped'] = df.apply(
        lambda row: row['other_llm_accuracy_capped']
        if row['other_llm_answer_type'] in ["Same", "Nothing"]
        else 1 - row['other_llm_accuracy_capped'],
        axis=1
    )

    df['advice_direction_final'] = np.where(
        np.abs(df['advice_direction']) < 0.001, 0,  # explicitly neutral remains neutral
        np.where(df['other_llm_answer'] == df['final_answer'], 1, -1)
    )

    if 'change_of_mind' not in df.columns:
        df['change_of_mind'] = (df['initial_answer'] != df['final_answer']).astype(int)

    new_columns = [
        'first_turn_confidence_initial_chosen',
        'second_turn_confidence_initial_chosen',
        'first_turn_confidence_final_chosen',
        'second_turn_confidence_final_chosen',
        'second_turn_binary_correctness',
        'advice_direction',
        'confirmation_flag',
        'shown_flag',
        'other_llm_accuracy_capped',
        'effective_other_llm_accuracy_capped',
        'advice_direction_final',
        'change_of_mind'
    ]

    return df

In [ ]:
def create_condition_encoding(df, categorical_columns, separator="_",
                            label_column="condition_label",
                            index_column="condition_idx",
                            return_mapping=True, verbose=False):


    df = df.copy()
    df[label_column] = df[categorical_columns].astype(str).agg(separator.join, axis=1)
    condition_labels = sorted(df[label_column].unique())

    label_to_index = {label: i for i, label in enumerate(condition_labels)}
    index_to_label = {i: label for label, i in label_to_index.items()}
    df[index_column] = df[label_column].map(label_to_index)

    if verbose:
        print(f"Created condition encoding from columns: {categorical_columns}")
        print(f"Number of unique conditions: {len(condition_labels)}")
        print("\nCondition mapping:")
        for label, idx in sorted(label_to_index.items(), key=lambda x: x[1]):
            count = (df[label_column] == label).sum()
            print(f"  {idx}: {label} (n={count})")

    if return_mapping:
        return df, label_to_index, index_to_label
    else:
        return df



In [ ]:
def create_train_test_split(df, n_train=10000, n_test=10000, train_seed=42, test_seed=123, verbose=False):

    total_requested = n_train + n_test
    if total_requested > len(df):
        raise ValueError(f"Requested {total_requested} samples ({n_train} train + {n_test} test) "
                        f"but only {len(df)} samples available in DataFrame")

    df_train = df.sample(n=n_train, random_state=train_seed)
    df_remaining = df.drop(df_train.index)
    if n_test > len(df_remaining):
        raise ValueError(f"Requested {n_test} test samples but only {len(df_remaining)} "
                        f"samples remaining after training split")

    df_test = df_remaining.sample(n=n_test, random_state=test_seed)
    df_remaining = df_remaining.drop(df_test.index)

    if verbose:
        print(f"Dataset split complete:")
        print(f"  - Original data: {len(df):,} samples")
        print(f"  - Training set: {len(df_train):,} samples")
        print(f"  - Test set: {len(df_test):,} samples")
        print(f"  - Remaining unused: {len(df_remaining):,} samples")
        print(f"  - Total used: {len(df_train) + len(df_test):,} samples "
              f"({(len(df_train) + len(df_test)) / len(df) * 100:.1f}%)")

    return df_train, df_test, df_remaining

In [ ]:
def evaluate_model_predictions(df, trace_latent, prediction_types=['initial', 'final', 'switch'],
                              n_options=4, save_figures=False, figure_path=None,
                              show_plots=True, verbose=True):

    def effective_advice_prob(advice_direction, advice_for_choice, advice_accuracy, n_options):
        return np.where(
            advice_direction == 0, 1.0 / n_options,
            np.where(advice_for_choice, advice_accuracy, (1 - advice_accuracy) / (n_options - 1))
        )

    def rescale_advice_prob(effective_advice_prob, advice_direction):
        return np.where(
            advice_direction == 1, (effective_advice_prob - 0.25) / 0.75,
            np.where(advice_direction == -1, (0.25 - effective_advice_prob) / 0.25, 0)
        )

    prior_conf_initial = df["first_turn_confidence_initial_chosen"].values
    prior_conf_final = df["first_turn_confidence_final_chosen"].values
    advice_accuracy = df["other_llm_accuracy_capped"].values
    advice_direction = df["advice_direction"].values
    other_llm_answer = df["other_llm_answer"].values
    final_answer = df["final_answer"].values
    initial_answer = df["initial_answer"].values
    shown_flag = df["shown_flag"].values

    # Compute effective advice probabilities
    advice_for_initial_choice = (other_llm_answer == initial_answer)
    advice_for_final_choice = (other_llm_answer == final_answer)

    effective_advice_prob_initial = effective_advice_prob(
        advice_direction, advice_for_initial_choice, advice_accuracy, n_options
    )
    effective_advice_prob_final = effective_advice_prob(
        advice_direction, advice_for_final_choice, advice_accuracy, n_options
    )

    # Advice direction final
    advice_direction_final = np.where(
        advice_direction == 0, 0,
        np.where(other_llm_answer == final_answer, 1, -1)
    )

    # Rescale probabilities
    effective_advice_prob_initial_rescaled = rescale_advice_prob(
        effective_advice_prob_initial, advice_direction
    )
    effective_advice_prob_final_rescaled = rescale_advice_prob(
        effective_advice_prob_final, advice_direction_final
    )


    all_param_list = [
        "intercept_final_conf",
        "intercept_initial_conf",
        "intercept_switch",
        "w_prior_shared",
        "w_shown_shared",
        # Condition-specific weights for initial
        "w_strength_initial_opposite_shown",
        "w_strength_initial_opposite_hidden",
        "w_strength_initial_same_shown",
        "w_strength_initial_same_hidden",
        "w_strength_initial_nothing",
        # Weights for final
        "w_strength_final_opposite",
        "w_strength_final_same",
        "w_strength_final_nothing",
        # Weights for switching
        "w_strength_COM_opposite",
        "w_strength_COM_same",
        "w_strength_COM_nothing"
    ]

    available_params = list(trace_latent.posterior.data_vars)
    params = {}
    for var in all_param_list:
        if var in available_params:
            params[var] = trace_latent.posterior[var].mean().values

    if verbose:
        print(f"Available parameters: {[p for p in all_param_list if p in params]}")


    results = {}


    for prediction_type in prediction_types:

        if prediction_type == 'initial':
            # Use condition-specific weights
            advice_strength = np.where(
                (advice_direction == 1) & (shown_flag == 1), params["w_strength_initial_same_shown"],
                np.where((advice_direction == 1) & (shown_flag == 0), params["w_strength_initial_same_hidden"],
                np.where((advice_direction == -1) & (shown_flag == 1), params["w_strength_initial_opposite_shown"],
                np.where((advice_direction == -1) & (shown_flag == 0), params["w_strength_initial_opposite_hidden"],
                params["w_strength_initial_nothing"])))
            )

            logit_post_conf = (
                params["intercept_initial_conf"] +
                params["w_prior_shared"] * prior_conf_initial +
                advice_strength * effective_advice_prob_initial_rescaled * advice_direction +
                params["w_shown_shared"] * shown_flag
            )

            predicted = expit(logit_post_conf)
            true_values = df["second_turn_confidence_initial_chosen"].values


            r = np.corrcoef(predicted, true_values)[0, 1]

            results['initial'] = {
                'predicted': predicted,
                'true': true_values,
                'correlation': r,
                'metric_name': 'Pearson r'
            }

            if verbose:
                print(f"\nPearson r (initial): {r:.3f}")


            if show_plots:
                plt.figure(figsize=(7, 5))
                plt.scatter(predicted, true_values, alpha=0.2)

                slope, intercept, _, _, _ = linregress(predicted, true_values)
                plt.plot([0, 1], [intercept, intercept + slope], color='black')
                plt.legend([f"Pearson's r = {r:.3f}"], loc='upper left', fontsize=12)

                plt.xlabel("Predicted Final Confidence in Initial Chosen", fontsize=14)
                plt.ylabel("Observed Final Confidence", fontsize=14)
                plt.xticks(fontsize=12)
                plt.yticks(fontsize=12)
                plt.xlim(0, 1)
                plt.ylim(0, 1)
                plt.tight_layout()

                if save_figures and figure_path:
                    plt.savefig(f'{figure_path}/predicted_vs_actual_initial_condition_specific.png', dpi=300)
                plt.show()

        elif prediction_type == 'final':
            # Simple weights for final
            advice_strength = np.where(
                advice_direction_final == 1, params["w_strength_final_same"],
                np.where(advice_direction_final == -1, params["w_strength_final_opposite"],
                params["w_strength_final_nothing"])
            )

            logit_post_conf = (
                params["intercept_final_conf"] +
                params["w_prior_shared"] * prior_conf_final +
                advice_strength * effective_advice_prob_final_rescaled * advice_direction_final +
                params["w_shown_shared"] * shown_flag
            )

            predicted = expit(logit_post_conf)
            true_values = df["second_turn_confidence_final_chosen"].values


            r = np.corrcoef(predicted, true_values)[0, 1]

            results['final'] = {
                'predicted': predicted,
                'true': true_values,
                'correlation': r,
                'metric_name': 'Pearson r'
            }

            if verbose:
                print(f"Pearson r (final): {r:.3f}")


            if show_plots:
                plt.figure(figsize=(7, 5))
                plt.scatter(predicted, true_values, alpha=0.2)

                slope, intercept, _, _, _ = linregress(predicted, true_values)
                plt.plot([0, 1], [intercept, intercept + slope], color='black')
                plt.legend([f"Pearson's r = {r:.3f}"], loc='upper left', fontsize=12)

                plt.xlabel("Predicted Final Confidence in Final Chosen", fontsize=14)
                plt.ylabel("Observed Final Confidence", fontsize=14)
                plt.xticks(fontsize=12)
                plt.yticks(fontsize=12)
                plt.xlim(0, 1)
                plt.ylim(0, 1)
                plt.tight_layout()

                if save_figures and figure_path:
                    plt.savefig(f'{figure_path}/predicted_vs_actual_final_condition_specific.png', dpi=300)
                plt.show()

        elif prediction_type == 'switch':
            # Switching model
            switch_flag = df["change_of_mind"].values

            advice_strength_switching = np.where(
                advice_direction == 1, params["w_strength_COM_same"],
                np.where(advice_direction == -1, params["w_strength_COM_opposite"],
                params["w_strength_COM_nothing"])
            )

            L = (
                params["intercept_switch"] +
                params["w_prior_shared"] * prior_conf_initial +
                advice_strength_switching * effective_advice_prob_initial_rescaled * advice_direction +
                params["w_shown_shared"] * shown_flag
            )

            p_switch = expit(-L)

            auc_score = roc_auc_score(switch_flag, p_switch)

            results['switch'] = {
                'predicted': p_switch,
                'true': switch_flag,
                'auc': auc_score,
                'metric_name': 'ROC AUC'
            }

            if verbose:
                print(f"\nAUC score: {auc_score:.3f}")


            if show_plots:
                plt.figure(figsize=(7, 5))
                sns.histplot(p_switch[switch_flag == 0], color="blue", label="Stay trials",
                           kde=True, stat="density", bins=np.linspace(0, 1, 30), alpha=0.6)
                sns.histplot(p_switch[switch_flag == 1], color="red", label="Switch trials",
                           kde=True, stat="density", bins=np.linspace(0, 1, 30), alpha=0.6)
                plt.axvline(0.5, linestyle="--", color="black", alpha=0.7)
                plt.xlabel("Predicted Change of Mind Rate", fontsize=14)
                plt.ylabel("Density", fontsize=14)
                plt.xticks(fontsize=12)
                plt.yticks(fontsize=12)
                plt.legend(["Stay trials", "Switch trials", f"ROC AUC: {auc_score:.3f}"],
                          fontsize=10, frameon=False, loc='upper right')

                sns.despine()
                plt.tight_layout()

                if save_figures and figure_path:
                    plt.savefig(f'{figure_path}/predicted_switching_histogram.png', dpi=300)
                plt.show()

    return results


# Plotting functions

In [ ]:
def plot_change_of_mind_opposite(df,
                                     conditions=None,
                                     accuracies=None,
                                     available_display_types=None,
                                     colors=None,
                                     figsize_per_condition=(8, 6),
                                     save_figure=False,
                                     save_path=None,
                                     filename='Change_of_Mind_Plot.png',
                                     dpi=300):

    if colors is None:
        colors = {'shown': 'tab:orange', 'hidden': 'tab:blue', 'hog': 'tab:green'}

    if available_display_types is None:
        available_display_types = ['shown', 'hidden']

    if conditions is None:
        conditions = ['Opposite']

    if accuracies is None:
        accuracies = sorted(df['other_llm_accuracy'].unique())

    fig, axes = plt.subplots(1, len(conditions),
                            figsize=(figsize_per_condition[0] * len(conditions), figsize_per_condition[1]),
                            squeeze=False)

    for j, condition in enumerate(conditions):
        ax = axes[0, j]

        bar_width = 0.35
        x = np.arange(len(accuracies))

        for idx, display_type in enumerate(available_display_types):
            sub_df = df[
                (df['initial_answer_display'] == display_type) &
                (df['other_llm_answer_type'] == condition)
            ]

            change_col = 'change_of_mind_hog' if display_type == 'hog' else 'change_of_mind'
            label = display_type.capitalize()

            means = sub_df.groupby('other_llm_accuracy')[change_col].mean() * 100
            sems = sub_df.groupby('other_llm_accuracy')[change_col].sem() * 100

            means = means.reindex(accuracies, fill_value=np.nan)
            sems = sems.reindex(accuracies, fill_value=0)

            ax.bar(
                x + idx * bar_width,
                means,
                bar_width,
                yerr=sems,
                capsize=5,
                label=label,
                color=colors[display_type],
                alpha=0.8,
                edgecolor='black'
            )

        display_condition = 'Neutral' if condition == 'Nothing' else condition
        ax.set_title(f'{display_condition} Advice')

        ax.set_xlabel('LLM Accuracy (%)')
        ax.set_ylabel('Change of Mind (%)')
        ax.set_xticks(x + bar_width / 2)
        ax.set_xticklabels(accuracies)
        ax.set_ylim(-5, 105)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['bottom'].set_linewidth(1.2)
        ax.spines['left'].set_linewidth(1.2)
        ax.legend(title="Answer")

    plt.tight_layout()


    if save_figure and save_path:
        os.makedirs(save_path, exist_ok=True)
        fig.savefig(os.path.join(save_path, filename), dpi=dpi)

    plt.show()

    return fig, axes

In [ ]:
def plot_change_of_mind_by_condition(df,
                                     conditions=None,
                                     accuracies=None,
                                     available_display_types=None,
                                     colors=None,
                                     figsize_per_condition=(8, 6),
                                     save_figure=False,
                                     save_path=None,
                                     filename='Change_of_Mind_Plot.png',
                                     dpi=300):



    if colors is None:
        colors = {'shown': 'tab:orange', 'hidden': 'tab:blue', 'hog': 'tab:green'}

    if available_display_types is None:
        available_display_types = ['shown', 'hidden']

    if conditions is None:
        conditions = ['Opposite', 'Same', 'Nothing']

    if accuracies is None:
        accuracies = sorted(df['other_llm_accuracy'].unique())

    # Create figure
    fig, axes = plt.subplots(1, len(conditions),
                            figsize=(figsize_per_condition[0] * len(conditions), figsize_per_condition[1]),
                            squeeze=False)

    for j, condition in enumerate(conditions):
        ax = axes[0, j]

        bar_width = 0.35
        x = np.arange(len(accuracies))

        for idx, display_type in enumerate(available_display_types):
            sub_df = df[
                (df['initial_answer_display'] == display_type) &
                (df['other_llm_answer_type'] == condition)
            ]

            change_col = 'change_of_mind_hog' if display_type == 'hog' else 'change_of_mind'
            label = display_type.capitalize()

            means = sub_df.groupby('other_llm_accuracy')[change_col].mean() * 100
            sems = sub_df.groupby('other_llm_accuracy')[change_col].sem() * 100

            means = means.reindex(accuracies, fill_value=np.nan)
            sems = sems.reindex(accuracies, fill_value=0)

            ax.bar(
                x + idx * bar_width,
                means,
                bar_width,
                yerr=sems,
                capsize=5,
                label=label,
                color=colors[display_type],
                alpha=0.8,
                edgecolor='black'
            )


        display_condition = 'Neutral' if condition == 'Nothing' else condition
        ax.set_title(f'{display_condition} Advice')

        ax.set_xlabel('LLM Accuracy (%)')
        ax.set_ylabel('Change of Mind (%)')
        ax.set_xticks(x + bar_width / 2)
        ax.set_xticklabels(accuracies)
        ax.set_ylim(-5, 105)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['bottom'].set_linewidth(1.2)
        ax.spines['left'].set_linewidth(1.2)
        ax.legend(title="Answer")

    plt.tight_layout()


    if save_figure and save_path:
        os.makedirs(save_path, exist_ok=True)
        fig.savefig(os.path.join(save_path, filename), dpi=dpi)

    plt.show()

    return fig, axes

In [ ]:
def plot_confidence_change_by_condition(df,
                                       conditions=None,
                                       accuracies=None,
                                       available_display_types=None,
                                       colors=None,
                                       figsize_per_condition=(8, 6),
                                       save_figure=False,
                                       save_path=None,
                                       filename='Confidence_Change_Plot.png',
                                       dpi=300,
                                       show_legend_on='first'):

    if colors is None:
        colors = {'shown': 'tab:orange', 'hidden': 'tab:blue', 'hog': 'tab:gray'}

    if available_display_types is None:
        available_display_types = ['shown', 'hidden']

    if conditions is None:
        conditions = ['Opposite', 'Same', 'Nothing']

    if accuracies is None:
        accuracies = sorted(df['other_llm_accuracy'].unique())

    fig, axes = plt.subplots(1, len(conditions),
                            figsize=(figsize_per_condition[0] * len(conditions), figsize_per_condition[1]),
                            squeeze=False)

    for idx_c, condition in enumerate(conditions):
        ax = axes[0, idx_c]

        bar_width = 0.35
        x = np.arange(len(accuracies))

        for idx_d, display_type in enumerate(available_display_types):
            sub_df = df[
                (df['initial_answer_display'] == display_type) &
                (df['other_llm_answer_type'] == condition)
            ].copy()

            sub_df['confidence_change'] = sub_df['second_turn_confidence_initial_chosen'] - sub_df['initial_confidence_chosen']

            means = sub_df.groupby('other_llm_accuracy')['confidence_change'].mean()
            sems = sub_df.groupby('other_llm_accuracy')['confidence_change'].sem()

            means = means.reindex(accuracies, fill_value=np.nan)
            sems = sems.reindex(accuracies, fill_value=0)

            ax.bar(
                x + idx_d * bar_width,
                means,
                bar_width,
                yerr=sems,
                capsize=5,
                label=display_type.capitalize(),
                color=colors[display_type],
                alpha=0.8,
                edgecolor='black'
            )


        display_condition = 'Neutral' if condition == 'Nothing' else condition
        ax.set_title(f'{display_condition} Advice')
        ax.set_xlabel('Advice Accuracy (%)')
        ax.set_ylabel('Confidence Change')
        ax.set_xticks(x + bar_width / 2)
        ax.set_xticklabels(accuracies)
        ax.axhline(0, color='gray', linestyle='--', linewidth=1)

        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['bottom'].set_linewidth(1.2)
        ax.spines['left'].set_linewidth(1.2)


        if (show_legend_on == 'first' and idx_c == 0) or \
           (show_legend_on == 'all') or \
           (isinstance(show_legend_on, int) and idx_c == show_legend_on):
            ax.legend(title="Display Type")

    plt.tight_layout()

    if save_figure and save_path:
        os.makedirs(save_path, exist_ok=True)
        fig.savefig(os.path.join(save_path, filename), dpi=dpi)

    plt.show()

    return fig, axes

In [ ]:
def plot_confidence_change_log_odds_by_condition(df,
                                                conditions=None,
                                                accuracies=None,
                                                available_display_types=None,
                                                colors=None,
                                                figsize_per_condition=(8, 6),
                                                save_figure=False,
                                                save_path=None,
                                                filename='LogOdds_Confidence_Change_Plot.png',
                                                dpi=300,
                                                show_legend_on='first'):

    if colors is None:
        colors = {'shown': 'tab:orange', 'hidden': 'tab:blue', 'hog': 'tab:gray'}

    if available_display_types is None:
        available_display_types = ['shown', 'hidden']

    if conditions is None:
        conditions = ['Opposite', 'Same', 'Nothing']

    if accuracies is None:
        accuracies = sorted(df['other_llm_accuracy'].unique())

    # Create composite plot
    fig, axes = plt.subplots(1, len(conditions),
                            figsize=(figsize_per_condition[0] * len(conditions), figsize_per_condition[1]),
                            squeeze=False)

    for idx_c, condition in enumerate(conditions):
        ax = axes[0, idx_c]

        bar_width = 0.35
        x = np.arange(len(accuracies))

        for idx_d, display_type in enumerate(available_display_types):
            sub_df = df[
                (df['initial_answer_display'] == display_type) &
                (df['other_llm_answer_type'] == condition)
            ].copy()

            sub_df['confidence_change_log_odds'] = sub_df['second_turn_confidence_initial_chosen_log_odds'] - sub_df['initial_confidence_chosen_log_odds']

            means = sub_df.groupby('other_llm_accuracy')['confidence_change_log_odds'].mean()
            sems = sub_df.groupby('other_llm_accuracy')['confidence_change_log_odds'].sem()

            means = means.reindex(accuracies, fill_value=np.nan)
            sems = sems.reindex(accuracies, fill_value=0)

            ax.bar(
                x + idx_d * bar_width,
                means,
                bar_width,
                yerr=sems,
                capsize=5,
                label=display_type.capitalize(),
                color=colors[display_type],
                alpha=0.8,
                edgecolor='black'
            )


        display_condition = 'Neutral' if condition == 'Nothing' else condition
        ax.set_title(f'{display_condition} Advice')
        ax.set_xlabel('Advice Accuracy (%)')
        ax.set_ylabel('Log Odds Confidence Change')
        ax.set_xticks(x + bar_width / 2)
        ax.set_xticklabels(accuracies)
        ax.axhline(0, color='gray', linestyle='--', linewidth=1)

        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['bottom'].set_linewidth(1.2)
        ax.spines['left'].set_linewidth(1.2)


        if (show_legend_on == 'first' and idx_c == 0) or \
           (show_legend_on == 'all') or \
           (isinstance(show_legend_on, int) and idx_c == show_legend_on):
            ax.legend(title="Display Type")

    plt.tight_layout()

    if save_figure and save_path:
        os.makedirs(save_path, exist_ok=True)
        fig.savefig(os.path.join(save_path, filename), dpi=dpi)

    plt.show()

    return fig, axes

In [ ]:
def plot_confidence_com_grid(df,
                            display_types_list=None,
                            conditions=None,
                            bin_width=0.05,
                            min_marker_size=10,
                            max_marker_size=400,
                            figsize_per_subplot=(6, 5),
                            save_figure=False,
                            save_path=None,
                            filename='confidence_COM_grid.png',
                            dpi=300,
                            marker_color='gray',
                            line_color='black',
                            marker_alpha=0.7):



    bins = np.arange(0, 1 + bin_width, bin_width)
    bin_midpoints = bins[:-1] + bin_width/2


    if display_types_list is None:
        display_types_list = sorted(df['initial_answer_display'].unique())

    if conditions is None:
        conditions = sorted(df['other_llm_answer_type'].unique())

    # Calculate global min/max for marker sizing across all data
    global_counts = []
    for disp_type in display_types_list:
        for condition in conditions:
            sub_df = df.loc[(df['initial_answer_display'] == disp_type) &
                            (df['other_llm_answer_type'] == condition)].copy()
            conf_col = 'initial_confidence_hog_chosen' if disp_type == 'hog' else 'initial_confidence_chosen'
            sub_df['confidence_bin'] = pd.cut(sub_df[conf_col], bins, include_lowest=True, labels=bin_midpoints)
            counts = sub_df['confidence_bin'].value_counts().reindex(bin_midpoints, fill_value=0)
            global_counts.extend(counts.values)

    global_min_count = np.min(global_counts) if global_counts else 0
    global_max_count = np.max(global_counts) if global_counts else 1


    fig, axes = plt.subplots(len(display_types_list), len(conditions),
                             figsize=(figsize_per_subplot[0] * len(conditions),
                                     figsize_per_subplot[1] * len(display_types_list)),
                             squeeze=False)

    for i, disp_type in enumerate(display_types_list):
        for j, condition in enumerate(conditions):
            ax = axes[i, j]

            sub_df = df.loc[(df['initial_answer_display'] == disp_type) &
                            (df['other_llm_answer_type'] == condition)].copy()

            conf_col = 'initial_confidence_hog_chosen' if disp_type == 'hog' else 'initial_confidence_chosen'
            com_col = 'change_of_mind_hog' if disp_type == 'hog' else 'change_of_mind'

            sub_df['confidence_bin'] = pd.cut(sub_df[conf_col], bins, include_lowest=True, labels=bin_midpoints)

            mean_data = sub_df.groupby('confidence_bin', observed=True)[com_col].mean() * 100
            counts = sub_df['confidence_bin'].value_counts().reindex(bin_midpoints, fill_value=0)

            mean_full = pd.Series(index=bin_midpoints, data=np.nan, dtype=float)
            mean_full.loc[mean_data.index.astype(float)] = mean_data.values

            # Global normalization of marker size
            if global_max_count > global_min_count:
                norm_counts = (counts - global_min_count) / (global_max_count - global_min_count)
            else:
                norm_counts = pd.Series(0.5, index=counts.index)

            sizes = min_marker_size + norm_counts * (max_marker_size - min_marker_size)

            ax.scatter(bin_midpoints, mean_full.values, s=sizes,
                      color=marker_color, edgecolor='black', alpha=marker_alpha)
            ax.plot(bin_midpoints, mean_full.values, linestyle='-', color=line_color)

            ax.set_xticks(np.arange(0, 1.1, 0.2))
            ax.set_yticks(np.arange(0, 101, 20))
            ax.set_xlabel('Confidence in Initial Chosen Option')
            ax.set_ylabel('Change of Mind Rate (%)')
            ax.set_xlim(0, 1)
            ax.set_ylim(-5, 105)


            display_label = 'Answer Hidden' if disp_type == 'hidden' else f'Answer {disp_type.capitalize()}'
            condition_label = 'Neutral' if condition == 'Nothing' else condition
            title = f"{display_label} - {condition_label} Advice"
            ax.set_title(title, fontsize=16, pad=12)

            sns.despine(ax=ax)

    plt.tight_layout()

    if save_figure and save_path:
        os.makedirs(save_path, exist_ok=True)
        fig.savefig(os.path.join(save_path, filename), dpi=dpi)

    plt.show()

    return fig, axes

In [ ]:
def plot_non_linear_threshold_OH_NH_COM_conf(df,
                                            focus_accuracies=None,
                                            min_bin_number=30,
                                            num_bins=15,
                                            save_figure=False,
                                            save_path=None,
                                            filename='sigmoid_plot_nothing_opp_hidden.png',
                                            figsize=(20, 8),
                                            curve_colors=None,
                                            font_size=18,
                                            show_summary=False):

    import warnings
    warnings.filterwarnings('ignore')

    if focus_accuracies is None:
        focus_accuracies = [50, 60, 70]

    if curve_colors is None:
        curve_colors = ['#9b59b6', '#3498db', '#e67e22']  # Purple, blue, orange

    # Define model functions
    def linear_func(x, a, b):
        return a * x + b

    def sigmoid_func_constrained(x, a, b, c):
        """Constrained sigmoid: always between 0 and a≤1"""
        return a / (1 + np.exp(-b * (x - c)))


    if show_summary:
        print("FITTING SIGMOID TO OPPOSITE HIDDEN DATA")
        print("="*60)

    opposite_hidden = df[(df['other_llm_answer_type'] == 'Opposite') &
                        (df['initial_answer_display'] == 'hidden') &
                        (df['other_llm_accuracy'].isin(focus_accuracies))].copy()

    model_results = []

    for accuracy in focus_accuracies:
        if show_summary:
            print(f"\nAnalyzing Opposite Hidden {accuracy}% accuracy...")

        data_subset = opposite_hidden[opposite_hidden['other_llm_accuracy'] == accuracy].copy()

        if len(data_subset) == 0:
            if show_summary:
                print(f"  No data for {accuracy}% accuracy")
            continue

        X = data_subset['initial_confidence_chosen'].values
        y = data_subset['change_of_mind'].values


        conf_bins = np.linspace(X.min(), X.max(), num_bins + 1)
        bin_centers = []
        bin_rates = []
        bin_counts = []

        for i in range(len(conf_bins)-1):
            mask = (X >= conf_bins[i]) & (X < conf_bins[i+1])
            bin_data = y[mask]
            if len(bin_data) >= min_bin_number:
                bin_centers.append((conf_bins[i] + conf_bins[i+1]) / 2)
                bin_rates.append(bin_data.mean())
                bin_counts.append(len(bin_data))

        bin_centers = np.array(bin_centers)
        bin_rates = np.array(bin_rates)
        bin_counts = np.array(bin_counts)

        weights = np.sqrt(bin_counts)

        results = {
            'accuracy': accuracy,
            'n_bins': len(bin_centers),
            'bin_centers': bin_centers,
            'bin_rates': bin_rates,
            'bin_sizes': bin_counts
        }


        try:
            bounds = ([0.01, -50, 0], [1.0, -0.1, 1])
            popt, _ = curve_fit(sigmoid_func_constrained, bin_centers, bin_rates,
                               p0=[0.8, -5, 0.5], sigma=1/weights,
                               bounds=bounds, maxfev=5000)

            results['sigmoid_params'] = popt
            if show_summary:
                print(f"  Sigmoid fit successful: a={popt[0]:.3f}, b={popt[1]:.3f}, c={popt[2]:.3f}")

        except Exception as e:
            if show_summary:
                print(f"  Sigmoid fitting failed: {e}")
            results['sigmoid_params'] = None

        model_results.append(results)

    if show_summary:
        print("\n\nFITTING LINEAR TO NOTHING HIDDEN DATA")
        print("="*60)

    nothing_hidden = df[(df['other_llm_answer_type'] == 'Nothing') &
                        (df['initial_answer_display'] == 'hidden') &
                        (df['other_llm_accuracy'].isin(focus_accuracies))].copy()

    nothing_hidden_results = []

    for accuracy in focus_accuracies:
        if show_summary:
            print(f"\nAnalyzing Nothing Hidden {accuracy}% accuracy...")

        data_subset = nothing_hidden[nothing_hidden['other_llm_accuracy'] == accuracy].copy()

        if len(data_subset) == 0:
            if show_summary:
                print(f"  No data for {accuracy}% accuracy")
            continue

        X = data_subset['initial_confidence_chosen'].values
        y = data_subset['change_of_mind'].values


        conf_bins = np.linspace(X.min(), X.max(), num_bins + 1)
        bin_centers = []
        bin_rates = []
        bin_counts = []

        for i in range(len(conf_bins)-1):
            mask = (X >= conf_bins[i]) & (X < conf_bins[i+1])
            bin_data = y[mask]
            if len(bin_data) >= min_bin_number:
                bin_centers.append((conf_bins[i] + conf_bins[i+1]) / 2)
                bin_rates.append(bin_data.mean())
                bin_counts.append(len(bin_data))

        bin_centers = np.array(bin_centers)
        bin_rates = np.array(bin_rates)
        bin_counts = np.array(bin_counts)

        weights = np.sqrt(bin_counts)

        results = {
            'accuracy': accuracy,
            'n_bins': len(bin_centers),
            'bin_centers': bin_centers,
            'bin_rates': bin_rates,
            'bin_sizes': bin_counts
        }


        try:
            linear_params, _ = curve_fit(linear_func, bin_centers, bin_rates,
                                        p0=[0, 0.5], sigma=1/weights)

            results['linear_params'] = linear_params
            if show_summary:
                print(f"  Linear fit successful: slope={linear_params[0]:.3f}, intercept={linear_params[1]:.3f}")

        except Exception as e:
            if show_summary:
                print(f"  Linear fitting failed: {e}")
            results['linear_params'] = None

        nothing_hidden_results.append(results)


    if show_summary:
        print("\n\nCreating comparison plot...")

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=figsize)
    plt.rcParams.update({'font.size': font_size})


    ax1.set_title('Answer Hidden Opposite Advice', fontsize=font_size + 2)

    for i, result in enumerate(model_results):
        if result.get('sigmoid_params') is not None:
            accuracy = result['accuracy']


            data_subset = opposite_hidden[opposite_hidden['other_llm_accuracy'] == accuracy]
            X_min = data_subset['initial_confidence_chosen'].min()
            X_max = data_subset['initial_confidence_chosen'].max()
            X_smooth = np.linspace(X_min, X_max, 200)


            sigmoid_curve = sigmoid_func_constrained(X_smooth, *result['sigmoid_params'])
            ax1.plot(X_smooth, sigmoid_curve, color=curve_colors[i], linewidth=5,
                    label=f"{accuracy}% Accuracy", alpha=0.9)


            bin_centers = result['bin_centers']
            bin_rates = result['bin_rates']
            bin_sizes = result['bin_sizes']

            if len(bin_centers) > 0:
                max_size = max(bin_sizes)
                min_size = min(bin_sizes)
                sizes = [50 + 450 * (s - min_size) / (max_size - min_size) if max_size > min_size else 200
                         for s in bin_sizes]

                ax1.scatter(bin_centers, bin_rates, color=curve_colors[i], s=sizes,
                           alpha=0.7, edgecolor='white', linewidth=1, zorder=5)


            a, b, c = result['sigmoid_params']
            if a > 0.5 and b != 0:
                conf_at_half = c - np.log(a/0.5 - 1)/b
                if 0 <= conf_at_half <= 1:
                    ax1.axvline(x=conf_at_half, color=curve_colors[i], linestyle='--',
                               linewidth=2, alpha=0.7)

    ax2.set_title('Answer Hidden Neutral Advice', fontsize=font_size + 2)

    for i, result in enumerate(nothing_hidden_results):
        if result.get('linear_params') is not None:
            accuracy = result['accuracy']


            data_subset = nothing_hidden[nothing_hidden['other_llm_accuracy'] == accuracy]
            X_min = data_subset['initial_confidence_chosen'].min()
            X_max = data_subset['initial_confidence_chosen'].max()
            X_smooth = np.linspace(X_min, X_max, 200)


            linear_curve = linear_func(X_smooth, *result['linear_params'])
            ax2.plot(X_smooth, linear_curve, color=curve_colors[i], linewidth=5,
                    label=f"{accuracy}% Accuracy", alpha=0.9)


            bin_centers = result['bin_centers']
            bin_rates = result['bin_rates']
            bin_sizes = result['bin_sizes']

            if len(bin_centers) > 0:
                max_size = max(bin_sizes)
                min_size = min(bin_sizes)
                sizes = [50 + 450 * (s - min_size) / (max_size - min_size) if max_size > min_size else 200
                         for s in bin_sizes]

                ax2.scatter(bin_centers, bin_rates, color=curve_colors[i], s=sizes,
                           alpha=0.7, edgecolor='white', linewidth=1, zorder=5)


            slope, intercept = result['linear_params']
            if slope != 0:
                conf_at_half = (0.5 - intercept) / slope
                if 0 <= conf_at_half <= 1:
                    ax2.axvline(x=conf_at_half, color=curve_colors[i], linestyle='--',
                               linewidth=2, alpha=0.7)

    for ax in [ax1, ax2]:
        ax.axhline(y=0.5, color='gray', linestyle=':', alpha=0.5, linewidth=2)

        ax.set_xlabel('Initial Confidence in Chosen Option', fontsize=font_size)
        ax.set_ylabel('Change of Mind Rate', fontsize=font_size)
        ax.legend(fontsize=font_size - 2)
        ax.set_ylim(0, 1.1)
        ax.set_xlim(0, 1)

        ax.set_yticks([0, 0.25, 0.5, 0.75, 1])
        ax.set_xticks([0, 0.2, 0.4, 0.6, 0.8, 1.0])

        ax.spines['bottom'].set_linewidth(2)
        ax.spines['left'].set_linewidth(2)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.tick_params(width=2)

    plt.tight_layout()


    if save_figure:
        if save_path:
            plt.savefig(save_path, dpi=300, bbox_inches='tight')
        else:
            plt.savefig(filename, dpi=300, bbox_inches='tight')

    plt.show()


    if show_summary:
        print("\n\nSUMMARY")
        print("="*60)
        print(f"Opposite Hidden - Successfully fitted: {sum(1 for r in model_results if r.get('sigmoid_params') is not None)} curves")
        print(f"Nothing Hidden - Successfully fitted: {sum(1 for r in nothing_hidden_results if r.get('linear_params') is not None)} curves")


        for accuracy in focus_accuracies:
            opp_count = len(opposite_hidden[opposite_hidden['other_llm_accuracy'] == accuracy])
            noth_count = len(nothing_hidden[nothing_hidden['other_llm_accuracy'] == accuracy])
            print(f"\n{accuracy}% accuracy data counts:")
            print(f"  Opposite Hidden: {opp_count} samples")
            print(f"  Nothing Hidden: {noth_count} samples")

    return fig, (ax1, ax2), model_results, nothing_hidden_results

In [ ]:
def plot_observed_vs_bayes_optimal_confidence(df,
                                             display_types=None,
                                             conditions=None,
                                             colors=None,
                                             bin_width=0.05,
                                             min_marker_size=10,
                                             max_marker_size=400,
                                             figsize_per_subplot=(6, 5),
                                             save_figure=False,
                                             save_path=None,
                                             filename='observed_final_confidence_composite_global_marker.png',
                                             dpi=300,
                                             show_diagonal=True,
                                             replace_nothing_with_neutral=True):

    if display_types is None:
        display_types = ['hidden', 'shown']

    if conditions is None:
        conditions = ['Nothing', 'Same', 'Opposite']

    if colors is None:
        colors = ['tab:blue', 'tab:orange', 'tab:green']

    bins = np.arange(0, 1 + bin_width, bin_width)
    bin_midpoints = bins[:-1] + bin_width/2

    # Calculate global min/max for marker sizing across all data
    global_counts = []
    for condition in conditions:
        for disp in display_types:
            sub_df = df.loc[
                (df['initial_answer_display'] == disp) &
                (df['other_llm_answer_type'] == condition)
            ].copy()
            bayes_col = 'bayes_optimal_probability_initial_chosen'
            sub_df['bayes_bin'] = pd.cut(sub_df[bayes_col], bins, include_lowest=True, labels=bin_midpoints)
            counts = sub_df['bayes_bin'].value_counts().reindex(bin_midpoints, fill_value=0)
            global_counts.extend(counts.values)

    global_min_count = np.min(global_counts) if global_counts else 0
    global_max_count = np.max(global_counts) if global_counts else 1

    fig, axes = plt.subplots(1, len(conditions),
                            figsize=(figsize_per_subplot[0] * len(conditions), figsize_per_subplot[1]))

    if len(conditions) == 1:
        axes = [axes]

    for j, condition in enumerate(conditions):
        ax = axes[j]

        for i, disp in enumerate(display_types):
            sub_df = df.loc[
                (df['initial_answer_display'] == disp) &
                (df['other_llm_answer_type'] == condition)
            ].copy()

            bayes_col = 'bayes_optimal_probability_initial_chosen'
            conf_col = 'second_turn_confidence_initial_chosen'
            label = disp.capitalize()

            sub_df['bayes_bin'] = pd.cut(sub_df[bayes_col], bins, include_lowest=True, labels=bin_midpoints)

            mean_conf = sub_df.groupby('bayes_bin', observed=True)[conf_col].mean()
            counts = sub_df['bayes_bin'].value_counts().reindex(bin_midpoints, fill_value=0)

            mean_full = pd.Series(index=bin_midpoints, data=np.nan, dtype=float)
            mean_full.loc[mean_conf.index.astype(float)] = mean_conf.values

            # Global normalization of marker size
            if global_max_count > global_min_count:
                norm_counts = (counts - global_min_count) / (global_max_count - global_min_count)
            else:
                norm_counts = pd.Series(0.5, index=counts.index)

            sizes = min_marker_size + norm_counts * (max_marker_size - min_marker_size)

            ax.scatter(bin_midpoints, mean_full.values, s=sizes,
                      color=colors[i], label=label, edgecolor='black', alpha=0.7)
            ax.plot(bin_midpoints, mean_full.values, linestyle='-', color=colors[i])


        if show_diagonal:
            ax.plot([0, 1], [0, 1], linestyle='--', color='gray')

        ax.set_xticks(np.arange(0, 1.1, 0.2))
        ax.set_yticks(np.arange(0, 1.1, 0.2))
        ax.set_xlabel('Bayes-optimal Probability')
        ax.set_xlim(0, 1.05)
        ax.set_ylim(0, 1.05)

        condition_label = condition
        if replace_nothing_with_neutral and condition == 'Nothing':
            condition_label = 'Neutral'
        ax.set_title(f'{condition_label} Advice')

        if j == 0:
            ax.set_ylabel('Observed Final Confidence')
            ax.legend(title='Answer')
        else:
            ax.set_ylabel('')

        sns.despine(ax=ax)

    plt.tight_layout()

    if save_figure and save_path:
        os.makedirs(save_path, exist_ok=True)
        fig.savefig(os.path.join(save_path, filename), dpi=dpi)

    plt.show()

    return fig, axes

In [ ]:
def plot_observed_vs_bayes_optimal_log_odds(df,
                                           x_limits=(-4, 4.75),
                                           y_limits=(-4, 4.75),
                                           display_types=None,
                                           conditions=None,
                                           colors=None,
                                           log_odds_bins=None,
                                           min_marker_size=10,
                                           max_marker_size=400,
                                           figsize_per_subplot=(6, 5),
                                           save_figure=False,
                                           save_path=None,
                                           filename='LogOdds_observed_final_log_odds_composite_global_marker.png',
                                           dpi=300,
                                           show_diagonal=True,
                                           show_grid=False,
                                           major_tick_spacing=2,
                                           minor_tick_spacing=1,
                                           replace_nothing_with_neutral=True,
                                           print_statistics=False):

    if display_types is None:
        display_types = ['hidden', 'shown']

    if conditions is None:
        conditions = ['Nothing', 'Same', 'Opposite']

    if colors is None:
        colors = ['tab:blue', 'tab:orange', 'tab:green']

    if log_odds_bins is None:
        log_odds_bins = np.linspace(-3, 3, 13)

    log_odds_bin_midpoints = log_odds_bins[:-1] + np.diff(log_odds_bins)/2

    if print_statistics:
        print(f"Using axis limits - X: {x_limits}, Y: {y_limits}")
        print(f"X-axis range: {x_limits[1] - x_limits[0]:.1f}")
        print(f"Y-axis range: {y_limits[1] - y_limits[0]:.1f}")

    # Calculate global min/max for marker sizing across all data
    global_counts = []
    for condition in conditions:
        for disp in display_types:
            sub_df = df.loc[
                (df['initial_answer_display'] == disp) &
                (df['other_llm_answer_type'] == condition)
            ].copy()

            bayes_col = 'bayes_optimal_log_odds_initial_chosen'
            sub_df['bayes_log_odds_bin'] = pd.cut(sub_df[bayes_col], log_odds_bins,
                                                   include_lowest=True, labels=log_odds_bin_midpoints)
            counts = sub_df['bayes_log_odds_bin'].value_counts().reindex(log_odds_bin_midpoints, fill_value=0)
            global_counts.extend(counts.values)

    global_min_count = np.min(global_counts) if global_counts else 0
    global_max_count = np.max(global_counts) if global_counts else 1

    fig, axes = plt.subplots(1, len(conditions),
                            figsize=(figsize_per_subplot[0] * len(conditions), figsize_per_subplot[1]))


    if len(conditions) == 1:
        axes = [axes]

    for j, condition in enumerate(conditions):
        ax = axes[j]

        for i, disp in enumerate(display_types):
            sub_df = df.loc[
                (df['initial_answer_display'] == disp) &
                (df['other_llm_answer_type'] == condition)
            ].copy()

            bayes_col = 'bayes_optimal_log_odds_initial_chosen'
            conf_col = 'second_turn_confidence_initial_chosen_log_odds'
            label = disp.capitalize()

            sub_df['bayes_log_odds_bin'] = pd.cut(sub_df[bayes_col], log_odds_bins,
                                                   include_lowest=True, labels=log_odds_bin_midpoints)


            mean_log_odds = sub_df.groupby('bayes_log_odds_bin', observed=True)[conf_col].mean()
            counts = sub_df['bayes_log_odds_bin'].value_counts().reindex(log_odds_bin_midpoints, fill_value=0)

            mean_full = pd.Series(index=log_odds_bin_midpoints, data=np.nan, dtype=float)
            mean_full.loc[mean_log_odds.index.astype(float)] = mean_log_odds.values

            # Global normalization of marker size
            if global_max_count > global_min_count:
                norm_counts = (counts - global_min_count) / (global_max_count - global_min_count)
            else:
                norm_counts = pd.Series(0.5, index=counts.index)

            sizes = min_marker_size + norm_counts * (max_marker_size - min_marker_size)

            ax.scatter(log_odds_bin_midpoints, mean_full.values, s=sizes, color=colors[i],
                       label=label, edgecolor='black', alpha=0.7)
            ax.plot(log_odds_bin_midpoints, mean_full.values, linestyle='-', color=colors[i])

        if show_diagonal:
            diag_min = max(x_limits[0], y_limits[0])
            diag_max = min(x_limits[1], y_limits[1])
            ax.plot([diag_min, diag_max], [diag_min, diag_max], linestyle='--', color='gray')

        ax.set_xlim(x_limits)
        ax.set_ylim(y_limits)

        if x_limits == y_limits:
            ax.set_aspect('equal', adjustable='box')

        major_x_ticks = np.arange(-10, 10, major_tick_spacing)
        major_y_ticks = np.arange(-10, 10, major_tick_spacing)


        minor_x_ticks = np.arange(-10, 10, minor_tick_spacing)
        minor_y_ticks = np.arange(-10, 10, minor_tick_spacing)

        major_x_ticks = major_x_ticks[(major_x_ticks >= x_limits[0]) & (major_x_ticks <= x_limits[1])]
        major_y_ticks = major_y_ticks[(major_y_ticks >= y_limits[0]) & (major_y_ticks <= y_limits[1])]
        minor_x_ticks = minor_x_ticks[(minor_x_ticks >= x_limits[0]) & (minor_x_ticks <= x_limits[1])]
        minor_y_ticks = minor_y_ticks[(minor_y_ticks >= y_limits[0]) & (minor_y_ticks <= y_limits[1])]


        ax.set_xticks(major_x_ticks)
        ax.set_yticks(major_y_ticks)
        ax.set_xticks(minor_x_ticks, minor=True)
        ax.set_yticks(minor_y_ticks, minor=True)


        ax.tick_params(axis='both', which='major', labelsize=14, length=6, width=1.5)
        ax.tick_params(axis='both', which='minor', length=3, width=1)

        ax.set_xlabel('Bayes-optimal Log Odds')


        if show_grid:
            ax.grid(True, alpha=0.3)

        condition_label = condition
        if replace_nothing_with_neutral and condition == 'Nothing':
            condition_label = 'Neutral'
        ax.set_title(f'{condition_label} Advice')

        if j == 0:
            ax.set_ylabel('Observed Final Log Odds')
            ax.legend(title='Answer')
        else:
            ax.set_ylabel('')

        sns.despine(ax=ax)

    plt.tight_layout()

    if save_figure and save_path:
        os.makedirs(save_path, exist_ok=True)
        fig.savefig(os.path.join(save_path, filename), dpi=dpi)

    plt.show()


    if print_statistics:
        all_x_values = []
        all_y_values = []

        for condition in conditions:
            for disp in display_types:
                sub_df = df.loc[
                    (df['initial_answer_display'] == disp) &
                    (df['other_llm_answer_type'] == condition)
                ].copy()

                if len(sub_df) > 0:
                    bayes_col = 'bayes_optimal_log_odds_initial_chosen'
                    conf_col = 'second_turn_confidence_initial_chosen_log_odds'


                    all_x_values.extend(sub_df[bayes_col].dropna().values)

                    all_y_values.extend(sub_df[conf_col].dropna().values)

        if all_x_values and all_y_values:

            print(f"\nData coverage statistics:")
            print(f"X-axis (Bayes optimal): min={np.min(all_x_values):.2f}, max={np.max(all_x_values):.2f}")
            print(f"Y-axis (Observed): min={np.min(all_y_values):.2f}, max={np.max(all_y_values):.2f}")

            x_captured = np.sum((np.array(all_x_values) >= x_limits[0]) & (np.array(all_x_values) <= x_limits[1]))
            y_captured = np.sum((np.array(all_y_values) >= y_limits[0]) & (np.array(all_y_values) <= y_limits[1]))
            x_percentage = (x_captured / len(all_x_values)) * 100
            y_percentage = (y_captured / len(all_y_values)) * 100

            print(f"\nPercentage of data captured by axis limits:")
            print(f"  X-axis {x_limits}: {x_percentage:.1f}% ({x_captured}/{len(all_x_values)} points)")
            print(f"  Y-axis {y_limits}: {y_percentage:.1f}% ({y_captured}/{len(all_y_values)} points)")
            print(f"  Points outside limits: X={len(all_x_values) - x_captured}, Y={len(all_y_values) - y_captured}")

    return fig, axes

In [ ]:
def plot_bayes_confidence_update_comparison(df,
                                           condition='Opposite',
                                           accuracy=80,
                                           display_type='hidden',
                                           bin_width=0.1,
                                           figsize=(10, 6),
                                           save_figure=False,
                                           save_path=None,
                                           filename=None,
                                           dpi=300,
                                           show_legend=True,
                                           ylabel='Confidence Change',
                                           xlabel='Initial Confidence'):


    bins = np.arange(0, 1 + bin_width, bin_width)
    bin_labels = [f'{b:.1f}' if b in np.arange(0, 1.1, 0.2) else '' for b in bins[:-1]]

    sub_df = df[
        (df['initial_answer_display'] == display_type) &
        (df['other_llm_answer_type'] == condition) &
        (df['other_llm_accuracy'] == accuracy)
    ].copy()

    if len(sub_df) == 0:
        print(f"No data found for {display_type} display, {condition} condition, {accuracy}% accuracy")
        return None, None

    if display_type == 'hog':
        sub_df['initial_confidence'] = sub_df['initial_confidence_hog_chosen']
        sub_df['bayes_optimal_prob'] = sub_df['bayes_optimal_probability_hog_chosen']
        sub_df['confidence_gap'] = sub_df['confidence_gap_hog_chosen']
    else:
        sub_df['initial_confidence'] = sub_df['initial_confidence_chosen']
        sub_df['bayes_optimal_prob'] = sub_df['bayes_optimal_probability_initial_chosen']
        sub_df['confidence_gap'] = sub_df['confidence_gap_initial_chosen']

    sub_df['bayes_update'] = sub_df['bayes_optimal_prob'] - sub_df['initial_confidence']
    sub_df['confidence_bin'] = pd.cut(sub_df['initial_confidence'], bins=bins, include_lowest=True)


    sub_df_melted = sub_df.melt(id_vars='confidence_bin',
                                value_vars=['bayes_update', 'confidence_gap'],
                                var_name='Measure',
                                value_name='Value')

    measure_names = {
        'bayes_update': 'Bayesian Update',
        'confidence_gap': 'Observed Update'
    }
    sub_df_melted['Measure'] = sub_df_melted['Measure'].map(measure_names)

    fig, ax = plt.subplots(figsize=figsize)
    sns.boxplot(x='confidence_bin', y='Value', hue='Measure', data=sub_df_melted, ax=ax)
    ax.set_xticks(np.arange(len(bins)-1))
    ax.set_xticklabels(bin_labels, rotation=0, fontsize=14)
    ax.axhline(0, color='gray', linestyle='--', alpha=0.5)

    ax.set_xlabel(xlabel, fontsize=16)
    ax.set_ylabel(ylabel, fontsize=16)
    ax.tick_params(axis='x', labelsize=14)
    ax.tick_params(axis='y', labelsize=14)
    ax.set_title(f'{display_type.capitalize()} Display - {condition} Advice - {accuracy}% Accuracy',
                 fontsize=18, pad=15)

    if show_legend:
        ax.legend(title='Measure', fontsize=14, title_fontsize=16)
    else:
        ax.legend().remove()

    sns.despine()

    plt.tight_layout()
    if save_figure:
        if save_path:
            plt.savefig(save_path, dpi=dpi)
        elif filename:
            plt.savefig(filename, dpi=dpi)
        else:

            auto_filename = f'bayes_confidence_update_{condition.lower()}{display_type}{accuracy}.png'
            plt.savefig(auto_filename, dpi=dpi)

    plt.show()

    return fig, ax

In [ ]:
def plot_bayes_update_grid_all_accuracies(df,
                                         condition='Opposite',
                                         display_type='hidden',
                                         bin_width=0.1,
                                         figsize=(16, 18),
                                         save_figure=False,
                                         save_path=None,
                                         filename=None,
                                         dpi=300,
                                         y_range=(-1, 0.2),
                                         y_tick_spacing=0.2):

    accuracies = sorted(df['other_llm_accuracy'].unique())
    bins = np.arange(0, 1 + bin_width, bin_width)
    bin_labels = [f'{b:.1f}' if b in np.arange(0, 1.1, 0.2) else '' for b in bins[:-1]]

    n_accuracies = len(accuracies)
    n_cols = 2
    n_rows = int(np.ceil(n_accuracies / n_cols))
    fig, axes = plt.subplots(n_rows, n_cols, figsize=figsize)
    axes = axes.flatten()

    handles, labels = None, None

    for i, (ax, selected_accuracy) in enumerate(zip(axes[:n_accuracies], accuracies)):
        sub_df = df[
            (df['initial_answer_display'] == display_type) &
            (df['other_llm_answer_type'] == condition) &
            (df['other_llm_accuracy'] == selected_accuracy)
        ].copy()

        if len(sub_df) == 0:
            ax.text(0.5, 0.5, f'No data for\n{selected_accuracy}% accuracy',
                    ha='center', va='center', transform=ax.transAxes)
            ax.set_xticks([])
            ax.set_yticks([])
            continue
        if display_type == 'hog':
            sub_df['initial_confidence'] = sub_df['initial_confidence_hog_chosen']
            sub_df['bayes_optimal_prob'] = sub_df['bayes_optimal_probability_hog_chosen']
            sub_df['confidence_gap'] = sub_df['confidence_gap_hog_chosen']
        else:
            sub_df['initial_confidence'] = sub_df['initial_confidence_chosen']
            sub_df['bayes_optimal_prob'] = sub_df['bayes_optimal_probability_initial_chosen']
            sub_df['confidence_gap'] = sub_df['confidence_gap_initial_chosen']

        sub_df['bayes_update'] = sub_df['bayes_optimal_prob'] - sub_df['initial_confidence']
        sub_df['confidence_bin'] = pd.cut(sub_df['initial_confidence'], bins=bins, include_lowest=True)

        sub_df_melted = sub_df.melt(id_vars='confidence_bin',
                                    value_vars=['bayes_update', 'confidence_gap'],
                                    var_name='Measure',
                                    value_name='Value')

        measure_names = {
            'bayes_update': 'Bayesian Update',
            'confidence_gap': 'Observed Update'
        }
        sub_df_melted['Measure'] = sub_df_melted['Measure'].map(measure_names)
        sns.boxplot(x='confidence_bin', y='Value', hue='Measure', data=sub_df_melted, ax=ax)
        ax.axhline(0, linestyle=':', color='gray')

        ax.spines['bottom'].set_linewidth(2)
        ax.spines['left'].set_linewidth(2)
        ax.spines['bottom'].set_color('black')
        ax.spines['left'].set_color('black')

        ax.tick_params(axis='both', which='major', width=2, length=7, labelsize=14, colors='black')
        ax.set_xlabel('Initial Confidence', fontsize=16)
        ax.set_ylabel('Confidence Change', fontsize=16)

        ax.set_xticks(np.arange(len(bin_labels)))
        ax.set_xticklabels(bin_labels, fontsize=14)

        y_ticks = np.arange(y_range[0], y_range[1] + y_tick_spacing, y_tick_spacing)
        ax.set_yticks(y_ticks)
        ax.set_yticklabels([f'{tick:.1f}' for tick in y_ticks], fontsize=14)
        ax.set_ylim(y_range[0] - 0.05, y_range[1] + 0.05)

        ax.grid(False)

        if handles is None:
            handles, labels = ax.get_legend_handles_labels()

        ax.legend().remove()

        ax.set_title(f'Accuracy = {selected_accuracy}%', fontsize=16)

    for j in range(n_accuracies, len(axes)):
        axes[j].set_visible(False)

    sns.despine()
    fig.legend(handles, labels, loc='upper right', fontsize=14, title='Measure', title_fontsize=16)

    main_title = f'{display_type.capitalize()} Display - {condition} Advice'
    fig.suptitle(main_title, fontsize=20, y=0.995)

    plt.tight_layout(rect=[0, 0, 0.9, 0.99])

    if save_figure:
        if save_path:
            plt.savefig(save_path, dpi=dpi)
        elif filename:
            plt.savefig(filename, dpi=dpi)
        else:

            auto_filename = f'bayes_update_grid_{condition.lower()}_{display_type}_all_accuracies.png'
            plt.savefig(auto_filename, dpi=dpi)

    plt.show()

    return fig, axes

In [ ]:
def plot_answer_wrong_COM_composite(df,
                                   available_display_types=None,
                                   conditions=None,
                                   accuracies=None,
                                   colors=None,
                                   figsize_per_condition=(8, 6),
                                   save_figure=False,
                                   save_path=None,
                                   filename='Answer_Wrong_COM_Composite.png',
                                   dpi=300,
                                   ylabel='Change of Initial Answer Rate (%)',
                                   hog_label='Wrong',
                                   bar_width=0.35,
                                   ylim=(-5, 105)):

    if colors is None:
        colors = {'shown': 'tab:orange', 'hidden': 'tab:blue', 'hog': 'tab:green'}

    if available_display_types is None:
        available_display_types = ['shown', 'hog']

    if conditions is None:
        conditions = df['other_llm_answer_type'].unique()

    if accuracies is None:
        accuracies = sorted(df['other_llm_accuracy'].unique())

    fig, axes = plt.subplots(1, len(conditions),
                            figsize=(figsize_per_condition[0] * len(conditions), figsize_per_condition[1]),
                            squeeze=False)

    for j, condition in enumerate(conditions):
        ax = axes[0, j]

        x = np.arange(len(accuracies))

        for idx, display_type in enumerate(available_display_types):
            sub_df = df[
                (df['initial_answer_display'] == display_type) &
                (df['other_llm_answer_type'] == condition)
            ]

            change_col = 'change_of_mind_hog' if display_type == 'hog' else 'change_of_mind'
            if display_type == 'hog':
                label = hog_label
            else:
                label = display_type.capitalize()

            means = sub_df.groupby('other_llm_accuracy')[change_col].mean() * 100
            sems = sub_df.groupby('other_llm_accuracy')[change_col].sem() * 100

            means = means.reindex(accuracies, fill_value=np.nan)
            sems = sems.reindex(accuracies, fill_value=0)

            ax.bar(
                x + idx * bar_width,
                means,
                bar_width,
                yerr=sems,
                capsize=5,
                label=label,
                color=colors[display_type],
                alpha=0.8,
                edgecolor='black'
            )

        display_condition = 'Neutral' if condition == 'Nothing' else condition
        ax.set_title(f'{display_condition} Advice')

        ax.set_xlabel('LLM Accuracy (%)')
        ax.set_ylabel(ylabel)
        ax.set_xticks(x + bar_width / 2)
        ax.set_xticklabels(accuracies)
        ax.set_ylim(ylim[0], ylim[1])
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['bottom'].set_linewidth(1.2)
        ax.spines['left'].set_linewidth(1.2)
        ax.legend(title="Answer")

    plt.tight_layout()

    if save_figure and save_path:
        os.makedirs(save_path, exist_ok=True)
        fig.savefig(os.path.join(save_path, filename), dpi=dpi)

    plt.show()

    return fig, axes

In [ ]:
def plot_for_composite_CSB_COM_acrossmodels(df,
                                        figsize=(8, 6), dpi=300, ylim=(-5, 100)):
    sns.set(style="white", context="talk")

    df = df.copy()
    df['initial_answer_display_norm'] = df['initial_answer_display'].astype(str).str.strip().str.lower()
    df['other_llm_answer_type_norm'] = df['other_llm_answer_type'].astype(str).str.strip().str.capitalize()

    display_types = ['shown', 'hidden']
    display_labels = {'shown': 'Shown', 'hidden': 'Hidden'}
    conditions = ['Opposite', 'Same', 'Nothing']

    means = {dt: [] for dt in display_types}
    sems  = {dt: [] for dt in display_types}

    for cond in conditions:
        for dt in display_types:
            sub = df[
                (df['initial_answer_display_norm'] == dt) &
                (df['other_llm_answer_type_norm'] == cond)
            ]

            vals = sub['change_of_mind'] * 100
            m = vals.mean() if len(vals) else np.nan
            se = vals.sem(ddof=1) if len(vals) > 1 else 0.0
            means[dt].append(m)
            sems[dt].append(se)


    fig, ax = plt.subplots(figsize=figsize)
    x = np.arange(len(conditions))
    bar_width = 0.35

    ax.bar(x - bar_width/2, means['shown'], bar_width,
           yerr=sems['shown'], capsize=5, label=display_labels['shown'],
           edgecolor='black', linewidth=0.6, alpha=0.9, color='tab:orange')

    ax.bar(x + bar_width/2, means['hidden'], bar_width,
           yerr=sems['hidden'], capsize=5, label=display_labels['hidden'],
           edgecolor='black', linewidth=0.6, alpha=0.9, color='tab:blue')

    ax.set_xticks(x)
    ax.set_xticklabels(conditions)
    ax.set_xlabel('Advice Condition')
    ax.set_ylabel('Change of Mind (%)')
    ax.set_ylim(ylim)
    ax.legend(frameon=False, title='Initial Answer Visibility')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    plt.tight_layout()


    return fig, ax

In [ ]:
def plot_for_composite_CSB_conf_acrossmodels(df, figsize=(8, 6), ylim=None):

    sns.set(style="white", context="talk")


    df = df.copy()
    df['initial_answer_display_norm'] = df['initial_answer_display'].astype(str).str.strip().str.lower()
    df['other_llm_answer_type_norm'] = df['other_llm_answer_type'].astype(str).str.strip().str.capitalize()

    display_types = ['shown', 'hidden']
    display_labels = {'shown': 'Shown', 'hidden': 'Hidden'}
    conditions = ['Opposite', 'Same', 'Nothing']


    df['confidence_change'] = df['second_turn_confidence_initial_chosen'] - df['initial_confidence_chosen']


    means = {dt: [] for dt in display_types}
    sems  = {dt: [] for dt in display_types}

    for cond in conditions:
        for dt in display_types:
            sub = df[
                (df['initial_answer_display_norm'] == dt) &
                (df['other_llm_answer_type_norm'] == cond)
            ]
            vals = sub['confidence_change']
            m = vals.mean() if len(vals) else np.nan
            se = vals.sem(ddof=1) if len(vals) > 1 else 0.0
            means[dt].append(m)
            sems[dt].append(se)


    fig, ax = plt.subplots(figsize=figsize)
    x = np.arange(len(conditions))
    bar_width = 0.35

    ax.bar(x - bar_width/2, means['shown'], bar_width,
           yerr=sems['shown'], capsize=5, label=display_labels['shown'],
           edgecolor='black', linewidth=0.6, alpha=0.9, color='tab:orange')

    ax.bar(x + bar_width/2, means['hidden'], bar_width,
           yerr=sems['hidden'], capsize=5, label=display_labels['hidden'],
           edgecolor='black', linewidth=0.6, alpha=0.9, color='tab:blue')

    ax.set_xticks(x)

    condition_labels = ['Opposite', 'Same', 'Neutral']
    ax.set_xticklabels(condition_labels)
    ax.set_xlabel('Advice Condition')
    ax.set_ylabel('Confidence Change')
    ax.axhline(0, color='gray', linestyle='--', linewidth=1)

    if ylim is not None:
        ax.set_ylim(ylim)

    ax.legend(frameon=False, title='Initial Answer Visibility')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    plt.tight_layout()

    return fig, ax

In [ ]:
def plot_obs_bayes_pred_COM_allmodels(df, figsize=(8, 6), print_stats=False, ylim=(-5, 105)):

    sns.set(style="white", context="talk")

    def calculate_bayes_com(row):
        probs = [
            row['bayes_optimal_probability_1'],
            row['bayes_optimal_probability_2'],
            row['bayes_optimal_probability_3'],
            row['bayes_optimal_probability_4']
        ] if 'bayes_optimal_probability_1' in row else [
            row['bayes_optimal_probability_a'],
            row['bayes_optimal_probability_b']
        ]

        initial_prob = row['bayes_optimal_probability_initial_chosen']
        max_prob = max(probs)

        return int(initial_prob < max_prob - 1e-10)

    df = df.copy()
    if 'bayes_com' not in df.columns:
        df['bayes_com'] = df.apply(calculate_bayes_com, axis=1)

    colors = {'shown': 'tab:orange', 'hidden': 'tab:blue', 'hog': 'tab:gray'}
    available_display_types = [dt for dt in ['shown', 'hidden', 'hog']
                               if dt in df['initial_answer_display'].unique()]
    condition = 'Opposite'
    accuracies = sorted(df['other_llm_accuracy'].unique())


    fig, ax = plt.subplots(figsize=figsize)


    num_display_types = len(available_display_types)
    bar_width = 0.8 / (num_display_types * 2)
    x = np.arange(len(accuracies))

    for idx, display_type in enumerate(available_display_types):
        sub_df = df[
            (df['initial_answer_display'] == display_type) &
            (df['other_llm_answer_type'] == condition)
        ]

        change_col = 'change_of_mind_hog' if display_type == 'hog' else 'change_of_mind'

        observed_means = []
        observed_sems = []
        bayes_means = []
        bayes_sems = []

        for acc in accuracies:
            acc_data = sub_df[sub_df['other_llm_accuracy'] == acc]
            if len(acc_data) > 0:
                obs_mean = acc_data[change_col].mean() * 100
                obs_sem = acc_data[change_col].sem() * 100 if len(acc_data) > 1 else 0
                observed_means.append(obs_mean)
                observed_sems.append(obs_sem)

                bayes_mean = acc_data['bayes_com'].mean() * 100
                bayes_sem = acc_data['bayes_com'].sem() * 100 if len(acc_data) > 1 else 0
                bayes_means.append(bayes_mean)
                bayes_sems.append(bayes_sem)
            else:
                observed_means.append(np.nan)
                observed_sems.append(0)
                bayes_means.append(np.nan)
                bayes_sems.append(0)

        observed_means = np.array(observed_means)
        observed_sems = np.array(observed_sems)
        bayes_means = np.array(bayes_means)
        bayes_sems = np.array(bayes_sems)

        bar_offset = (idx * 2) * bar_width - (num_display_types - 1) * bar_width


        ax.bar(
            x + bar_offset,
            observed_means,
            bar_width,
            yerr=observed_sems,
            capsize=5,
            label=f'{display_type.capitalize()} (Observed)',
            color=colors[display_type],
            alpha=0.8,
            edgecolor='black'
        )


        ax.bar(
            x + bar_offset + bar_width,
            bayes_means,
            bar_width,
            yerr=bayes_sems,
            capsize=5,
            label=f'{display_type.capitalize()} (Bayes Optimal)',
            color=colors[display_type],
            alpha=0.4,
            edgecolor='black',
            hatch='//'
        )

    ax.set_xlabel('Advice Accuracy (%)')
    ax.set_ylabel('Change of Mind (%)')
    ax.set_xticks(x)
    ax.set_xticklabels(accuracies)
    ax.set_ylim(ylim)

    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_linewidth(1.2)
    ax.spines['left'].set_linewidth(1.2)

    ax.grid(axis='y', alpha=0.3, linestyle='--')

    plt.tight_layout()



    return fig, ax

In [ ]:
def plot_obs_bayes_pred_confidence_allmodels(df, figsize=(8, 6), ylim=(-0.9, 0.3)):

    sns.set(style="white", context="talk")


    df = df.copy()
    df['bayes_confidence_gap'] = df['bayes_optimal_probability_initial_chosen'] - df['initial_confidence_chosen']


    colors = {'shown': 'tab:orange', 'hidden': 'tab:blue', 'hog': 'tab:gray'}
    available_display_types = [dt for dt in ['shown', 'hidden', 'hog']
                               if dt in df['initial_answer_display'].unique()]
    condition = 'Opposite'
    accuracies = sorted(df['other_llm_accuracy'].unique())


    fig, ax = plt.subplots(figsize=figsize)


    num_display_types = len(available_display_types)
    bar_width = 0.8 / (num_display_types * 2)
    x = np.arange(len(accuracies))

    for idx, display_type in enumerate(available_display_types):
        sub_df = df[
            (df['initial_answer_display'] == display_type) &
            (df['other_llm_answer_type'] == condition)
        ].copy()

        sub_df['confidence_change'] = sub_df['second_turn_confidence_initial_chosen'] - sub_df['initial_confidence_chosen']


        observed_means = []
        observed_sems = []
        bayes_means = []
        bayes_sems = []

        for acc in accuracies:
            acc_data = sub_df[sub_df['other_llm_accuracy'] == acc]
            if len(acc_data) > 0:

                obs_mean = acc_data['confidence_change'].mean()
                obs_sem = acc_data['confidence_change'].sem() if len(acc_data) > 1 else 0
                observed_means.append(obs_mean)
                observed_sems.append(obs_sem)

                bayes_mean = acc_data['bayes_confidence_gap'].mean()
                bayes_sem = acc_data['bayes_confidence_gap'].sem() if len(acc_data) > 1 else 0
                bayes_means.append(bayes_mean)
                bayes_sems.append(bayes_sem)
            else:
                observed_means.append(np.nan)
                observed_sems.append(0)
                bayes_means.append(np.nan)
                bayes_sems.append(0)

        observed_means = np.array(observed_means)
        observed_sems = np.array(observed_sems)
        bayes_means = np.array(bayes_means)
        bayes_sems = np.array(bayes_sems)


        bar_offset = (idx * 2) * bar_width - (num_display_types - 1) * bar_width


        ax.bar(
            x + bar_offset,
            observed_means,
            bar_width,
            yerr=observed_sems,
            capsize=5,
            label=f'{display_type.capitalize()} (Observed)',
            color=colors[display_type],
            alpha=0.8,
            edgecolor='black'
        )


        ax.bar(
            x + bar_offset + bar_width,
            bayes_means,
            bar_width,
            yerr=bayes_sems,
            capsize=5,
            label=f'{display_type.capitalize()} (Bayes Optimal)',
            color=colors[display_type],
            alpha=0.4,
            edgecolor='black',
            hatch='//'
        )

    ax.set_xlabel('Advice Accuracy (%)')
    ax.set_ylabel('Confidence Gap')
    ax.set_xticks(x)
    ax.set_xticklabels(accuracies)
    ax.axhline(0, color='gray', linestyle='--', linewidth=1)
    ax.set_ylim(ylim)

    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_linewidth(1.2)
    ax.spines['left'].set_linewidth(1.2)

    ax.grid(axis='y', alpha=0.3, linestyle='--')

    plt.tight_layout()

    return fig, ax

# Function: Over/Underconfidence Score (OUCS)

In [ ]:
def calc_overunderconfidence_score(df,
                                  second_choice_var="initial",
                                  expt_type=None,
                                  collapse_accuracy=True,
                                  split_by_initial_correctness=False,
                                  split_by_change_of_mind=False,
                                  split_by_change_of_mind_hog=False,
                                  bins=None):

    if 'change_of_mind' not in df.columns:
        df['change_of_mind'] = (df['initial_answer'] != df['final_answer']).astype(int)

    if expt_type == "hog" and 'change_of_mind_hog' not in df.columns:
        df['change_of_mind_hog'] = (df['final_answer'] != df['initial_answer_hog']).astype(int)

    if bins is None:
        bins = np.arange(0, 1.05, 0.05)
    bin_midpoints = bins[:-1] + np.diff(bins)/2

    def get_columns(display_type):
        if display_type == 'hog':
            return {
                'bayes': "bayes_optimal_probability_hog_chosen",
                'option': "second_turn_confidence_hog_chosen"
            }
        else:
            return {
                'bayes': f"bayes_optimal_probability_{second_choice_var}_chosen",
                'option': f"second_turn_confidence_{second_choice_var}_chosen"
            }

    def calculate_score(binned_stats, bin_midpoints, total_count):
        score = 0
        for index, row in binned_stats.iterrows():
            avg_confidence = row['mean']
            bin_midpoint = bin_midpoints[index]
            bin_count = row['count']
            if not np.isnan(avg_confidence) and total_count > 0:
                weight = bin_count / total_count
                score += (avg_confidence - bin_midpoint) * weight
        return score
    display_types = ['hidden', 'shown']
    if split_by_change_of_mind_hog:
        display_types = ['hog']

    answer_types = ['Nothing', 'Same', 'Opposite']
    results_list = []

    if split_by_initial_correctness:
        split_col = 'initial_binary_correctness'
        split_values = [0, 1]
    elif split_by_change_of_mind:
        split_col = 'change_of_mind'
        split_values = [0, 1]
    elif split_by_change_of_mind_hog:
        split_col = 'change_of_mind_hog'
        split_values = [0, 1]
    else:
        split_col = None
        split_values = [None]

    for split_value in split_values:
        for answer_type in answer_types:
            for display_type in display_types:

                cols = get_columns(display_type)
                if split_col is not None:
                    subset_df = df[
                        (df['other_llm_answer_type'] == answer_type) &
                        (df['initial_answer_display'] == display_type) &
                        (df[split_col] == split_value)
                    ].copy()
                else:
                    subset_df = df[
                        (df['other_llm_answer_type'] == answer_type) &
                        (df['initial_answer_display'] == display_type)
                    ].copy()

                if subset_df.empty:
                    continue

                if len(subset_df) <= 1:
                    subset_df[cols['bayes']] = subset_df[cols['bayes']].fillna(0)


                subset_df['bayes_prob_bin'] = pd.cut(subset_df[cols['bayes']],
                                                     bins=bins,
                                                     include_lowest=True,
                                                     right=False)
                binned_stats = subset_df.groupby('bayes_prob_bin', observed=False)[cols['option']].agg(
                    ['mean', 'sem', 'count']
                ).reset_index()
                total_count = len(subset_df)
                score = calculate_score(binned_stats, bin_midpoints, total_count)

                results_list.append({
                    'Answer Type': answer_type,
                    'Display Type': display_type,
                    'Score': round(score, 4)
                })

    results_df = pd.DataFrame(results_list)
    print(results_df.to_string(index=False))

    return results_df

# CODE TO CREATE FIGURES AND TABLES

## Gemma 3 12B

In [ ]:
expt_type = "latitude"
file_path = f"{the_dir}gemma12b_latitude_experiment_data.csv"
df = pd.read_csv(file_path) #

In [ ]:
df_no_nans = find_nan_in_df(df)
df = df_no_nans

In [ ]:
df = add_new_confidence_columns(df, expt_type)
df = add_new_confidence_columns_log_odds_space(df, expt_type)
df = add_bayes_optimal_columns(df)
df = add_bayes_optimal_columns_log_odds_space(df)

**FIGURE 2A**

In [ ]:
fig, axes = plot_change_of_mind_by_condition(df)

**FIGURE 2B**

In [ ]:
fig, axes = plot_confidence_change_by_condition(df)

**FIGURE 9**

In [ ]:
fig, axes = plot_confidence_change_log_odds_by_condition(df)

**FIGURE 3**

In [ ]:
fig, axes = plot_confidence_com_grid(df)

**Figure 4**

In [ ]:
fig, ax = plot_for_composite_CSB_COM_acrossmodels(df)
fig, ax = plot_for_composite_CSB_conf_acrossmodels(df)

**Figure 5**

In [ ]:
figs = plot_obs_bayes_pred_COM_allmodels(df)
fig, ax = plot_obs_bayes_pred_confidence_allmodels(df)

**Extended Data Figure 1**

In [ ]:
fig, axes, sigmoid_results, linear_results = plot_non_linear_threshold_OH_NH_COM_conf(df)

**FIGURE 2C**

In [ ]:
fig, axes = plot_observed_vs_bayes_optimal_confidence(df)

**Extended Data Figure 2B**

In [ ]:
fig, axes = plot_observed_vs_bayes_optimal_log_odds(df)

**TABLE 1**

In [ ]:
results = calc_overunderconfidence_score(df, expt_type=expt_type)

**FIGURE 2D**

In [ ]:
# Plot Opposite Hidden at 80% accuracy
fig, ax = plot_bayes_confidence_update_comparison(
    df,
    condition='Opposite',
    accuracy=80,
    display_type='hidden',
  )

# Plot Same Hidden at 80% accuracy
fig, ax = plot_bayes_confidence_update_comparison(
    df,
    condition='Same',
    accuracy=80,
    display_type='hidden',
)


**Extended Data Figures 3 and 4**

In [ ]:
# Opposite Hidden
fig, axes = plot_bayes_update_grid_all_accuracies(
    df,
    condition='Opposite',
    display_type='hidden',
    y_range=(-1.2, 0.4),
    y_tick_spacing=0.2,
    figsize=(16, 20)
)

# For Same Hidden
fig, axes = plot_bayes_update_grid_all_accuracies(
    df,
    condition='Same',
    display_type='hidden',
    y_range=(0, 1),
    y_tick_spacing=0.2,
    figsize=(16, 20)
)

**Extended Data Figure 5B**

In [ ]:
expt_type = "hog" #note "hog" is Answer Wrong experiment
file_path = f"{the_dir}gemma12b_answerwrong_experiment_data.csv"
df = pd.read_csv(file_path)
df_no_nans = find_nan_in_df(df)
df = df_no_nans

fig, axes = plot_answer_wrong_COM_composite(
    df
)


**FIGURE 6A**

In [ ]:
expt_type = "latitude"
file_path = f"{the_dir}gemma12b_latitude_otherllm_experiment_data.csv"
df = pd.read_csv(file_path)
df_no_nans = find_nan_in_df(df)
df = df_no_nans
df = add_new_confidence_columns(df, expt_type)
fig, ax = plot_for_composite_CSB_COM_acrossmodels(df)
fig, ax =plot_for_composite_CSB_conf_acrossmodels(df)

**Figure 6B**

In [ ]:
expt_type = "latitude"
file_path = f"{the_dir}Gemma12B_multiturn_latitude_experiment_data.csv"
df = pd.read_csv(file_path)
df_no_nans = find_nan_in_df(df)
df = df_no_nans
df = add_new_confidence_columns(df, expt_type)
fig, ax = plot_for_composite_CSB_COM_acrossmodels(df)
fig, ax =plot_for_composite_CSB_conf_acrossmodels(df)


**Extended Data Figure 5A**

In [ ]:
expt_type = "latitude"
file_path = f"{the_dir}Gemma12B_likelihood_latitude_experiment_data.csv"
df = pd.read_csv(file_path)
df_no_nans = find_nan_in_df(df)
df = df_no_nans
df = add_new_confidence_columns(df, expt_type)
fig, ax = plot_for_composite_CSB_COM_acrossmodels(df)
fig, ax =plot_for_composite_CSB_conf_acrossmodels(df)

**Extended Data Figure 5C**

In [ ]:
expt_type = "latitude"
file_path = f"{the_dir}gemma12b_latitude_ICX_experiment_data.csv"
df = pd.read_csv(file_path)

df_no_nans = find_nan_in_df(df)
df = df_no_nans
df = df.rename(columns={'change_of_mind_actual':'change_of_mind'})
fig, axes = plot_change_of_mind_by_condition(df)

## Gemma 3 27B

In [ ]:
expt_type = "latitude"
file_path = f"{the_dir}gemma27b_latitude_experiment_data.csv"
df = pd.read_csv(file_path)

In [ ]:
df_no_nans = find_nan_in_df(df)
df = df_no_nans

In [ ]:
df = add_new_confidence_columns(df, expt_type)
df = add_bayes_optimal_columns(df)

**Figures 4 and 5**

In [ ]:
fig, ax = plot_for_composite_CSB_COM_acrossmodels(df)
fig, ax = plot_for_composite_CSB_conf_acrossmodels(df)
figs = plot_obs_bayes_pred_COM_allmodels(df)
fig, ax = plot_obs_bayes_pred_confidence_allmodels(df)

**Extended Data Figure 8A**

In [ ]:
fig, axes = plot_change_of_mind_by_condition(df)

**Extended Data Figure 8B**

In [ ]:
fig, axes = plot_confidence_change_by_condition(df)

**Extended Data Figure 8C**

In [ ]:
fig, axes = plot_observed_vs_bayes_optimal_confidence(df)

**Extended Data Table 7**

In [ ]:
results = calc_overunderconfidence_score(df, expt_type=expt_type)

**Extended Data Figure 8D**

In [ ]:
# Plot Opposite Hidden at 80% accuracy
fig, ax = plot_bayes_confidence_update_comparison(
    df,
    condition='Opposite',
    accuracy=80,
    display_type='hidden',
  )

# Plot Same Hidden at 80% accuracy
fig, ax = plot_bayes_confidence_update_comparison(
    df,
    condition='Same',
    accuracy=80,
    display_type='hidden',
)


## GPT4o

In [ ]:
expt_type = "latitude"
file_path = f"{the_dir}GPT4o_difficult_latitude_experiment_data.csv"
df = pd.read_csv(file_path)


In [ ]:
df_no_nans = find_nan_in_df(df)
df = df_no_nans

In [ ]:
df = add_new_confidence_columns(df, expt_type)
df = add_bayes_optimal_columns(df)

**Figures 4 and 5**

In [ ]:
fig, ax = plot_for_composite_CSB_COM_acrossmodels(df)
fig, ax = plot_for_composite_CSB_conf_acrossmodels(df)
figs = plot_obs_bayes_pred_COM_allmodels(df)
fig, ax = plot_obs_bayes_pred_confidence_allmodels(df)

**Extended Data Figure 9A**

In [ ]:
fig, axes = plot_change_of_mind_by_condition(df)

**Extended Data Figure 9B**

In [ ]:
fig, axes = plot_confidence_change_by_condition(df)

**Extended Data Figure 9C**

In [ ]:
fig, axes = plot_observed_vs_bayes_optimal_confidence(df)

**Extended Data Table 8**

In [ ]:
results = calc_overunderconfidence_score(df, expt_type=expt_type)

**Extended Data Figure 9D**

In [ ]:
# Plot Opposite Hidden at 80% accuracy
fig, ax = plot_bayes_confidence_update_comparison(
    df,
    condition='Opposite',
    accuracy=80,
    display_type='hidden',
  )

# Plot Same Hidden at 80% accuracy
fig, ax = plot_bayes_confidence_update_comparison(
    df,
    condition='Same',
    accuracy=80,
    display_type='hidden',
)


## GPTo1-preview

In [ ]:
expt_type = "latitude"
file_path = f"{the_dir}GPT4o1preview_difficult_latitude_experiment_data.csv"
df = pd.read_csv(file_path)


In [ ]:
df_no_nans = find_nan_in_df(df)
df = df_no_nans

**Extended Data Figure 10**

In [ ]:
fig, axes = plot_change_of_mind_by_condition(df)

## Llama 70B Instruct

In [ ]:
expt_type = "latitude"
file_path = f"{the_dir}llama70b_latitude_experiment_data.csv"
df = pd.read_csv(file_path)
df_no_nans = find_nan_in_df(df)
df = df_no_nans
df = add_new_confidence_columns(df, expt_type)
df = add_bayes_optimal_columns(df)

**Figure 4**

In [ ]:
fig, ax = plot_for_composite_CSB_COM_acrossmodels(df)
fig, ax = plot_for_composite_CSB_conf_acrossmodels(df)

**Figure 5**

In [ ]:
figs = plot_obs_bayes_pred_COM_allmodels(df)
fig, ax = plot_obs_bayes_pred_confidence_allmodels(df)

## DeepSeek 671B

In [ ]:
expt_type = "latitude"
file_path = f"{the_dir}DeepSeek_671_latitude_experiment_data.csv"
df = pd.read_csv(file_path)

In [ ]:
df_no_nans = find_nan_in_df(df)
df = df_no_nans
df = add_new_confidence_columns(df, expt_type)
df = add_bayes_optimal_columns(df)

**Figure 4**

In [ ]:
fig, ax = plot_for_composite_CSB_COM_acrossmodels(df)
fig, ax = plot_for_composite_CSB_conf_acrossmodels(df)

**Figure 5**

In [ ]:
figs = plot_obs_bayes_pred_COM_allmodels(df)
fig, ax = plot_obs_bayes_pred_confidence_allmodels(df)